In [ ]:
# === DELFIN Dashboard (Local Server) ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import json
import py3Dmol
import os
import subprocess
import shutil
import html
import re
import base64
import sys
import signal
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta

# Output widget for JavaScript execution (works in Voila)
_js_output = widgets.Output()

def _run_js(script):
    """Execute JavaScript in a way that works in both Jupyter and Voila."""
    if not script:
        return
    with _js_output:
        clear_output(wait=True)
        display(Javascript(script))


# RDKit for SMILES conversion
from rdkit import Chem
from rdkit.Chem import AllChem

# stk for metal complexes (optional)
try:
    import stk
    HAS_STK = True
except ImportError:
    HAS_STK = False

# Ensure local DELFIN is importable in notebooks
_deflin_root = None
for _base in [Path.cwd(), *Path.cwd().parents]:
    if (_base / 'delfin' / '__init__.py').exists():
        _deflin_root = _base
        break
    if (_base / 'delfin').is_dir() and (_base / 'delfin' / '__init__.py').exists():
        _deflin_root = _base
        break
if _deflin_root and str(_deflin_root) not in sys.path:
    sys.path.insert(0, str(_deflin_root))

# Force reload to pick up any code changes
import importlib
import delfin.config
import delfin.common.control_validator
importlib.reload(delfin.common.control_validator)
importlib.reload(delfin.config)
from delfin.config import validate_control_text

# === Paths (Local Server) ===
HOME = Path.home()

# Notebook directory
try:
    import IPython
    NOTEBOOK_DIR = Path(IPython.extract_module_locals()[1]['__vsc_ipynb_file__']).parent
except:
    NOTEBOOK_DIR = Path.cwd()

# Calc directory: always ~/calc
CALC_DIR = HOME / "calc"
BASE_DIR = CALC_DIR
CALC_DIR.mkdir(parents=True, exist_ok=True)

# ORCA: fixed path
ORCA_BASE_SELECTED = "/opt/orca"

# Run script: in same directory as this notebook
RUN_SCRIPT = NOTEBOOK_DIR / 'run_local.sh'

JOB_TIME_LIMITS = {
    '12h': '12:00:00',
    '24h': '24:00:00',
    '36h': '36:00:00',
    '48h': '48:00:00',
    '60h': '60:00:00',
}


def _resolve_time_limit(toggle_widget, custom_widget, default='48:00:00'):
    """Resolve time limit from toggle buttons or custom input field."""
    toggle_val = toggle_widget.value
    if toggle_val == 'Custom':
        try:
            hours = int(custom_widget.value)
            if hours < 1:
                raise ValueError
            return f'{hours}:00:00'
        except (ValueError, TypeError):
            return default
    return JOB_TIME_LIMITS.get(toggle_val, default)



# ========================================================================
# LOCAL JOB MANAGER with Queue (replaces Slurm sbatch/squeue/scancel)
# ========================================================================

import threading
import time as _time

_JOBS_FILE = HOME / '.delfin_jobs.json'
_NEXT_JOB_ID_KEY = '_next_job_id'

# Server resource limits
SERVER_MAX_CORES = 384
SERVER_MAX_RAM_MB = 1_400_000  # ~1.4 TB (leaves headroom from 1.5 TB)

# Queue lock for thread safety
_queue_lock = threading.Lock()


def _load_jobs():
    """Load jobs from JSON file."""
    if _JOBS_FILE.exists():
        try:
            data = json.loads(_JOBS_FILE.read_text())
            return data
        except (json.JSONDecodeError, IOError):
            return {_NEXT_JOB_ID_KEY: 1001, 'jobs': []}
    return {_NEXT_JOB_ID_KEY: 1001, 'jobs': []}


def _save_jobs(data):
    """Save jobs to JSON file."""
    _JOBS_FILE.write_text(json.dumps(data, indent=2, default=str))


def _update_job_status(job):
    """Update a single job's status by checking PID and exit code file."""
    if job['status'] not in ('RUNNING',):
        return job

    pid = job.get('pid')
    if pid is None:
        job['status'] = 'FAILED'
        return job

    # Check if process is still running
    try:
        os.kill(pid, 0)
        return job  # Still running
    except ProcessLookupError:
        pass  # Process gone
    except PermissionError:
        return job  # Can't signal but exists

    # Process finished - read exit code
    job_dir = job.get('job_dir', '')
    job_id = job.get('job_id', 0)
    exit_code_file = Path(job_dir) / f'.exit_code_{job_id}'

    if exit_code_file.exists():
        try:
            code = int(exit_code_file.read_text().strip())
            if code == 0:
                job['status'] = 'COMPLETED'
            elif code == 124:
                job['status'] = 'TIMEOUT'
            else:
                job['status'] = 'FAILED'
        except (ValueError, IOError):
            job['status'] = 'FAILED'
    else:
        job['status'] = 'FAILED'

    return job


def _get_running_resources():
    """Calculate total cores and RAM used by RUNNING jobs."""
    data = _load_jobs()
    total_cores = 0
    total_ram_mb = 0
    for job in data.get('jobs', []):
        if job['status'] == 'RUNNING':
            total_cores += job.get('pal', 0)
            total_ram_mb += job.get('pal', 0) * job.get('maxcore', 0)
    return total_cores, total_ram_mb


def _parse_time_to_seconds(time_str):
    """Convert HH:MM:SS to seconds."""
    parts = time_str.strip().split(':')
    try:
        if len(parts) == 3:
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        elif len(parts) == 2:
            return int(parts[0]) * 60 + int(parts[1])
        else:
            return int(parts[0]) * 3600
    except (ValueError, IndexError):
        return 48 * 3600


class _SubmitResult:
    """Mimics subprocess.CompletedProcess for compatibility."""
    def __init__(self, returncode, stdout='', stderr=''):
        self.returncode = returncode
        self.stdout = stdout
        self.stderr = stderr


def _start_job(job, data):
    """Actually start a PENDING job (called by queue worker)."""
    timeout_secs = _parse_time_to_seconds(job.get('time_limit', '48:00:00'))

    env = os.environ.copy()
    env['DELFIN_MODE'] = job['mode']
    env['DELFIN_JOB_NAME'] = job['name']
    env['DELFIN_JOB_ID'] = str(job['job_id'])
    if ORCA_BASE_SELECTED:
        env['DELFIN_ORCA_BASE'] = ORCA_BASE_SELECTED
    if job.get('inp_file'):
        env['DELFIN_INP_FILE'] = job['inp_file']
    if job.get('override'):
        env['DELFIN_OVERRIDE'] = job['override']
    if job.get('build_mult') is not None:
        env['BUILD_MULTIPLICITY'] = str(job['build_mult'])

    try:
        log_file = Path(job['job_dir']) / f'delfin_{job["job_id"]}.out'
        log_fd = open(log_file, 'w')

        proc = subprocess.Popen(
            ['timeout', str(timeout_secs), 'bash', str(RUN_SCRIPT)],
            cwd=str(job['job_dir']),
            env=env,
            stdout=log_fd,
            stderr=subprocess.STDOUT,
            start_new_session=True
        )

        job['pid'] = proc.pid
        job['pgid'] = os.getpgid(proc.pid)
        job['status'] = 'RUNNING'
        job['start_time'] = datetime.now().isoformat()
        job['log_file'] = str(log_file)
        return True
    except Exception as e:
        job['status'] = 'FAILED'
        job['error'] = str(e)
        return False


def _try_start_pending_jobs():
    """Check queue and start PENDING jobs if resources are available."""
    with _queue_lock:
        data = _load_jobs()
        jobs = data.get('jobs', [])

        # First update all RUNNING jobs
        for job in jobs:
            if job['status'] == 'RUNNING':
                _update_job_status(job)

        # Calculate currently used resources
        used_cores = sum(j.get('pal', 0) for j in jobs if j['status'] == 'RUNNING')
        used_ram = sum(j.get('pal', 0) * j.get('maxcore', 0) for j in jobs if j['status'] == 'RUNNING')

        # Try to start pending jobs (FIFO order)
        started_any = False
        for job in jobs:
            if job['status'] != 'PENDING':
                continue

            needed_cores = job.get('pal', 40)
            needed_ram = job.get('pal', 40) * job.get('maxcore', 6000)

            if (used_cores + needed_cores <= SERVER_MAX_CORES and
                    used_ram + needed_ram <= SERVER_MAX_RAM_MB):
                if _start_job(job, data):
                    used_cores += needed_cores
                    used_ram += needed_ram
                    started_any = True

        _save_jobs(data)
        return started_any


# === Queue Worker Thread ===
_queue_worker_running = True


def _queue_worker():
    """Background thread that checks the queue every 5 seconds."""
    while _queue_worker_running:
        try:
            _try_start_pending_jobs()
        except Exception:
            pass
        _time.sleep(5)


# Start queue worker
_queue_thread = threading.Thread(target=_queue_worker, daemon=True, name='delfin-queue')
_queue_thread.start()


def local_submit(job_dir, mode, job_name, inp_file=None,
                 time_limit='48:00:00', override=None, build_mult=None,
                 pal=40, maxcore=6000):
    """
    Enqueue a job (replaces sbatch).
    Job starts as PENDING and the queue worker starts it when resources are free.
    """
    if not RUN_SCRIPT.exists():
        return _SubmitResult(1, stderr=f'Run script not found: {RUN_SCRIPT}')

    with _queue_lock:
        data = _load_jobs()
        job_id = data.get(_NEXT_JOB_ID_KEY, 1001)
        data[_NEXT_JOB_ID_KEY] = job_id + 1

        job_entry = {
            'job_id': job_id,
            'name': job_name,
            'mode': mode,
            'pid': None,
            'pgid': None,
            'status': 'PENDING',
            'submit_time': datetime.now().isoformat(),
            'start_time': None,
            'job_dir': str(job_dir),
            'time_limit': time_limit,
            'pal': pal,
            'maxcore': maxcore,
            'inp_file': inp_file,
            'override': override,
            'build_mult': build_mult,
            'log_file': None
        }

        data['jobs'].append(job_entry)
        _save_jobs(data)

    # Immediately try to start (don't wait for the 5s tick)
    _try_start_pending_jobs()

    return _SubmitResult(0, stdout=f'Submitted local job {job_id}')


def _cancel_job(job_id):
    """Cancel a running or pending job."""
    with _queue_lock:
        data = _load_jobs()
        for job in data['jobs']:
            if job['job_id'] == job_id:
                if job['status'] == 'PENDING':
                    job['status'] = 'CANCELLED'
                    _save_jobs(data)
                    # Try to start next pending job
                    return True, f'Pending job {job_id} cancelled'
                elif job['status'] != 'RUNNING':
                    return False, f'Job {job_id} is not running (status: {job["status"]})'
                pgid = job.get('pgid')
                pid = job.get('pid')
                try:
                    if pgid:
                        os.killpg(pgid, signal.SIGTERM)
                    elif pid:
                        os.kill(pid, signal.SIGTERM)
                    job['status'] = 'CANCELLED'
                    _save_jobs(data)
                    return True, f'Job {job_id} cancelled'
                except ProcessLookupError:
                    job['status'] = 'CANCELLED'
                    _save_jobs(data)
                    return True, f'Job {job_id} already finished, marked cancelled'
                except Exception as e:
                    return False, f'Error cancelling job {job_id}: {e}'
        return False, f'Job {job_id} not found'



# === SMILES Conversion Functions ===

from delfin.smiles_converter import (
    smiles_to_xyz as _delfin_smiles_to_xyz,
    is_smiles_string as _delfin_is_smiles_string,
    contains_metal,
)

def smiles_to_xyz(smiles):
    """
    Wrapper around delfin.smiles_converter.smiles_to_xyz to preserve dashboard API.
    Returns: (xyz_string, num_atoms, method, error)
    """
    xyz_string, error = _delfin_smiles_to_xyz(smiles)
    if error:
        return None, 0, None, error
    num_atoms = sum(1 for line in xyz_string.splitlines() if line.strip())
    method = 'delfin.smiles_converter'
    return xyz_string, num_atoms, method, None

def is_smiles(text):
    """Check if text is a SMILES string."""
    try:
        return bool(_delfin_is_smiles_string(text))
    except Exception:
        return False

def clean_input_data(input_text):
    """
    Process input data: SMILES or XYZ coordinates.
    Returns: (cleaned_text, input_type)
    """
    text = input_text.strip()

    if not text:
        return '', 'empty'

    if is_smiles(text):
        return text, 'smiles'

    lines = text.split('\n')
    if len(lines) < 2:
        return text, 'xyz'

    first_line = lines[0].strip()
    try:
        atom_count = int(first_line)
        cleaned_lines = lines[2:]
        return '\n'.join(cleaned_lines).strip(), 'xyz'
    except ValueError:
        return text, 'xyz'


def parse_resource_settings(control_text):
    """Parse PAL and maxcore from CONTROL.txt content."""
    pal_match = re.search(r'^\s*PAL\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    maxcore_match = re.search(r'^\s*maxcore\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    pal = int(pal_match.group(1)) if pal_match else None
    maxcore = int(maxcore_match.group(1)) if maxcore_match else None
    return pal, maxcore


DEFAULT_CONTROL = """input_file=input.txt
NAME=
SMILES=
charge=[CHARGE]
------------------------------------
Solvation:
implicit_solvation_model=CPCM
solvent=[SOLVENT]
XTB_SOLVATOR=no
number_explicit_solv_molecules=2
------------------------------------
Global geometry optimisation:
xTB_method=XTB2
XTB_OPT=no
XTB_GOAT=no
CREST=no
multiplicity_global_opt=
------------------------------------
IMAG=yes
IMAG_scope=initial
IMAG_option=2
allow_imaginary_freq=0
IMAG_sp_energy_window=1e-3
IMAG_optimize_candidates=no
------------------------------------
calc_prop_of_interest=no
properties_of_interest=IP,EA
------------------------------------
Redox steps:
calc_initial=yes
oxidation_steps=1,2,3
reduction_steps=1,2,3
method=classic|manually|OCCUPIER
calc_potential_method=2
------------------------------------
ESD module (excited state dynamics):
ESD_modul=no
ESD_modus=TDDFT|deltaSCF|hybrid1
ESD_frequency=yes
states=S1,T1,S2,T2
ISCs=S1>T1,T1>S1
ICs=S2>S1
emission_rates=f,p
phosp_IROOT=1,2,3
phosp_keywords=
fluor_keywords=
TROOTSSL=-1,0,1
addition_S0=
DOHT=TRUE
ESD_LINES=LORENTZ
ESD_LINEW=50
ESD_INLINEW=250
ESD_NPOINTS=131072
ESD_MAXTIME=12000
hybrid1_geom_MaxIter=60
--------------------
Electrical Properties:
elprop_Dipole=no
elprop_Quadrupole=no
elprop_Hyperpol=no
elprop_Polar=no
elprop_PolarVelocity=no
elprop_PolarDipQuad=no
elprop_PolarQuadQuad=no
--------------------
deltaSCF Settings:
deltaSCF_DOMOM=true
deltaSCF_PMOM=false
deltaSCF_keepinitialref=true
deltaSCF_SOSCFHESSUP=LSR1
deltaSCF_keywords=FreezeAndRelease
deltaSCF_maxiter=300
deltaSCF_SOSCFConvFactor=500
deltaSCF_SOSCFMaxStep=0.1
--------------------
TDDFT Settings:
TDDFT_TDDFT_maxiter=500
TDDFT_nroots=15
TDDFT_maxdim=30
TDDFT_TDA=FALSE
TDDFT_followiroot=true
TDDFT_SOC=false
------------------------------------
MANUALLY:
multiplicity_0=
additions_0=
additions_TDDFT=
additions_T1=
additions_S1=
multiplicity_ox1=
additions_ox1=
multiplicity_ox2=
additions_ox2=
multiplicity_ox3=
additions_ox3=
multiplicity_red1=
additions_red1=
multiplicity_red2=
additions_red2=
multiplicity_red3=
additions_red3=
------------------------------------
Level of Theory:
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
relativity=ZORA
aux_jk=def2/J
aux_jk_rel=SARC/J
main_basisset=def2-SVP
main_basisset_rel=ZORA-def2-SVP
metal_basisset=def2-TZVP
metal_basisset_rel=SARC-ZORA-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.3
geom_opt=OPT
freq_type=FREQ
initial_guess=PModel
temperature=298.15
maxiter=125
qmmm_option=QM/PBEH-3c
------------------------------------
Reference value:
E_ref=
------------------------------------
Literature_reference=
reference_CV=V Vs. Fc+/Fc
E_00_exp=
E_red_exp=
E_red_2_exp=
E_red_3_exp=
E_ox_exp=
E_ox_2_exp=
E_ox_3_exp=
*E_red_exp=
*E_ox_exp=
------------------------------------
Prints:
print_MOs=yes
print_Loewdin_population_analysis=no
------------------------------------
Resource Settings:
PAL=40
maxcore=6000
parallel_workflows=yes
pal_jobs=4
orca_parallel_strategy=auto
enable_job_timeouts=no
job_timeout_hours=36
opt_timeout_hours=14
frequency_timeout_hours=36
sp_timeout_hours=3
------------------------------------
Automatic Error Recovery & Retry:
enable_auto_recovery=yes
max_recovery_attempts=3
enable_adaptive_parallelism=yes
enable_performance_metrics=yes
------------------------------------
OCCUPIER-Settings:
--------------------
OCCUPIER_method=auto
OCCUPIER_tree=own
OWN_TREE_PURE_WINDOW=3
OWN_progressive_from=no
fob_equal_weights=yes
frequency_calculation_OCCUPIER=no
occupier_selection=tolerance
occupier_precision=3
occupier_epsilon=5e-4
clean_override_window_h=0.002
clean_quality_improvement=0.05
clean_quality_good=0.05
maxiter_occupier=125
geom_opt_OCCUPIER=OPT
pass_wavefunction=no
approximate_spin_projection_APMethod=2
--------------------
OCCUPIER_sequence_profiles:
-3,-2,-1,0,+1,+2,+3=[
even electron number:
even_seq = [
  {"index": 1, "m": 1, "BS": "",    "from": 0},
  {"index": 2, "m": 1, "BS": "1,1", "from": 1},
  {"index": 3, "m": 1, "BS": "2,2", "from": 2},
  {"index": 4, "m": 3, "BS": "",    "from": 1},
  {"index": 5, "m": 3, "BS": "3,1", "from": 4},
  {"index": 6, "m": 3, "BS": "4,2", "from": 5},
  {"index": 7, "m": 5, "BS": "",    "from": 4}
]
-------------------
odd electron number:
odd_seq = [
  {"index": 1, "m": 2, "BS": "",    "from": 0},
  {"index": 2, "m": 2, "BS": "2,1", "from": 1},
  {"index": 3, "m": 2, "BS": "3,2", "from": 2},
  {"index": 4, "m": 4, "BS": "",    "from": 1},
  {"index": 5, "m": 4, "BS": "4,1", "from": 4},
  {"index": 6, "m": 4, "BS": "5,2", "from": 5},
  {"index": 7, "m": 6, "BS": "",    "from": 4}
]
]
"""

# Only GOAT template (inline since file doesn't exist in repo)
ONLY_GOAT_TEMPLATE = """input_file=input.txt
NAME=
SMILES=
charge=[CHARGE]
------------------------------------
Solvation:
implicit_solvation_model=CPCM
solvent=[SOLVENT]
XTB_SOLVATOR=no
number_explicit_solv_molecules=0
------------------------------------
Global geometry optimisation:
xTB_method=XTB2
XTB_OPT=no
XTB_GOAT=yes
GOAT_SELECTION=UPHILLATOMS
CREST=no
multiplicity_global_opt=
------------------------------------
IMAG=no
------------------------------------
calc_prop_of_interest=no
------------------------------------
Redox steps:
calc_initial=yes
oxidation_steps=
reduction_steps=
method=classic
calc_potential_method=2
------------------------------------
ESD module (excited state dynamics):
ESD_modul=no
------------------------------------
Level of Theory:
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
relativity=ZORA
aux_jk=def2/J
aux_jk_rel=SARC/J
main_basisset=def2-SVP
main_basisset_rel=ZORA-def2-SVP
metal_basisset=def2-TZVP
metal_basisset_rel=SARC-ZORA-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.3
geom_opt=OPT
freq_type=FREQ
initial_guess=PModel
temperature=298.15
maxiter=125
qmmm_option=QM/PBEH-3c
------------------------------------
Reference value:
E_ref=
------------------------------------
Prints:
print_MOs=yes
print_Loewdin_population_analysis=no
------------------------------------
Resource Settings:
PAL=40
maxcore=6000
parallel_workflows=yes
pal_jobs=4
orca_parallel_strategy=auto
enable_job_timeouts=no
job_timeout_hours=36
opt_timeout_hours=14
frequency_timeout_hours=36
sp_timeout_hours=3
------------------------------------
Automatic Error Recovery & Retry:
enable_auto_recovery=yes
max_recovery_attempts=3
enable_adaptive_parallelism=yes
enable_performance_metrics=yes
------------------------------------
OCCUPIER-Settings:
--------------------
OCCUPIER_method=auto
OCCUPIER_tree=own
OWN_TREE_PURE_WINDOW=3
OWN_progressive_from=no
fob_equal_weights=yes
frequency_calculation_OCCUPIER=no
occupier_selection=tolerance
occupier_precision=3
occupier_epsilon=5e-4
clean_override_window_h=0.002
clean_quality_improvement=0.05
clean_quality_good=0.05
maxiter_occupier=125
geom_opt_OCCUPIER=OPT
pass_wavefunction=no
approximate_spin_projection_APMethod=2
"""

common_layout = widgets.Layout(width='500px')
common_style = {'description_width': 'initial'}

# === Widgets ===
job_name_widget = widgets.Text(
    value='',
    placeholder='e.g. Fe_Complex_Ox',
    description='Job Name:',
    layout=common_layout,
    style=common_style
)

job_type_widget = widgets.ToggleButtons(
    options=['12h', '24h', '36h', '48h', '60h', 'Custom'],
    value='48h',
    description='Time Limit:',
    style=common_style,
    button_style='',
    layout=widgets.Layout(width='550px')
)

custom_time_widget = widgets.BoundedIntText(
    value=72,
    min=1,
    max=720,
    step=1,
    description='Hours:',
    layout=widgets.Layout(width='180px', display='none'),
    style=common_style
)

def _toggle_custom_time_visibility(change):
    if change['new'] == 'Custom':
        custom_time_widget.layout.display = ''
    else:
        custom_time_widget.layout.display = 'none'

job_type_widget.observe(_toggle_custom_time_visibility, names='value')

coords_help = widgets.Label("Input: XYZ coordinates (header auto-removed) or SMILES string")
coords_widget = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates or SMILES:\n\nXYZ example:\n42\nComment\nFe  0.0  0.0  0.0\nC   1.5  0.0  0.0\n\nSMILES example:\nCCO or c1ccccc1',
    description='Input:',
    layout=widgets.Layout(width='500px', height='200px'),
    style=common_style
)

# Spacer
button_spacer = widgets.Label(value="", layout=widgets.Layout(height="10px"))

convert_smiles_button = widgets.Button(
    description='CONVERT SMILES',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

build_complex_button = widgets.Button(
    description='BUILD COMPLEX',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

smiles_batch_help = widgets.Label("Batch SMILES list: one per line 'Name;SMILES;key=value;key=value'")
smiles_batch_widget = widgets.Textarea(
    value='',
    placeholder='name;SMILES;key=value;...\nNi_1;[Ni];charge=2;solvent=water\nCo_1;[Co];charge=3',
    description='SMILES List:',
    layout=widgets.Layout(width='500px', height='160px'),
    style=common_style
)

submit_smiles_list_button = widgets.Button(
    description='SUBMIT SMILES LIST',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
smiles_batch_output = widgets.Output()

# SMILES List Preview Navigation
_smiles_preview_index = {'current': 0}

smiles_prev_button = widgets.Button(
    description='\u25c0',
    button_style='info',
    layout=widgets.Layout(width='35px')
)
smiles_next_button = widgets.Button(
    description='\u25b6',
    button_style='info',
    layout=widgets.Layout(width='35px')
)
smiles_preview_label = widgets.HTML(
    value='<span style="font-size:12px;">0 / 0</span>',
    layout=widgets.Layout(width='50px', margin='0 2px')
)

control_help = widgets.Label("CONTROL.txt - edit parameters as needed")
control_widget = widgets.Textarea(
    value=DEFAULT_CONTROL,
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='500px'),
    style=common_style
)

submit_button = widgets.Button(
    description='SUBMIT JOB',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

validate_button = widgets.Button(
    description='VALIDATE CONTROL',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

output_area = widgets.Output()
validate_output = widgets.Output()

mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #1976d2',
    width='560px',
    height='420px',
    overflow='hidden'
))

xyz_copy_btn = widgets.Button(
    description='\U0001f4cb Copy Coordinates',
    button_style='success',
    layout=widgets.Layout(width='150px'),
    disabled=True
)
xyz_copy_status = widgets.HTML(value='', layout=widgets.Layout(margin='0 0 0 6px'))
xyz_copy_row = widgets.HBox([
    xyz_copy_btn,
    xyz_copy_status
], layout=widgets.Layout(gap='6px', align_items='center'))


only_goat_label = widgets.Label("Only GOAT:")
only_goat_charge = widgets.IntText(
    value=0,
    description='Charge:',
    style=common_style,
    layout=widgets.Layout(width='160px')
)
only_goat_solvent = widgets.Text(
    value='',
    placeholder='e.g. water',
    description='Solvent:',
    style=common_style,
    layout=widgets.Layout(width='220px')
)
only_goat_submit_button = widgets.Button(
    description='SUBMIT ONLY GOAT',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
only_goat_output = widgets.Output()

# Cache for converted XYZ coordinates
_converted_xyz_cache = {'smiles': None, 'xyz': None}
_current_xyz_for_copy = {'content': None}


def update_molecule_view(change=None):
    """Update molecule visualization."""
    global _converted_xyz_cache, _current_xyz_for_copy

    with mol_output:
        clear_output()
        raw_input = coords_widget.value.strip()

        if not raw_input:
            print("Please enter XYZ coordinates or SMILES.")
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            _current_xyz_for_copy = {'content': None}
            xyz_copy_btn.disabled = True
            xyz_copy_status.value = ''
            return

        cleaned_data, input_type = clean_input_data(raw_input)

        if input_type == 'smiles':
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            _current_xyz_for_copy = {'content': None}
            xyz_copy_btn.disabled = True
            xyz_copy_status.value = ''
            print("SMILES erkannt. Bitte 'CONVERT SMILES' klicken.")
            return

        # XYZ coordinates - visualize directly
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        coords = cleaned_data
        lines = [l for l in coords.split('\n') if l.strip()]
        num_atoms = len(lines)
        xyz_data = f"{num_atoms}\nGenerated by widget\n{coords}"
        _current_xyz_for_copy = {'content': xyz_data}
        xyz_copy_btn.disabled = False
        xyz_copy_status.value = '<span style="color:#388e3c;">XYZ ready to copy</span>'
        mol_view = py3Dmol.view(width=560, height=420)
        mol_view.addModel(xyz_data, "xyz")
        mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
        mol_view.zoomTo()
        mol_view.show()


def on_xyz_copy(button):
    content = _current_xyz_for_copy.get('content') if _current_xyz_for_copy else None
    if not content:
        xyz_copy_status.value = '<span style="color:#d32f2f;">No XYZ to copy</span>'
        return
    escaped = content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = (
        "navigator.clipboard.writeText('" + escaped + "')"
        ".then(() => console.log('Copied XYZ'))"
        ".catch(err => console.error('Copy failed:', err));"
    )
    _run_js(js_code)
    xyz_copy_status.value = '<span style="color:#388e3c;">Copied to clipboard</span>'


xyz_copy_btn.on_click(on_xyz_copy)
coords_widget.observe(update_molecule_view, names='value')


def handle_convert_smiles(button):
    global _converted_xyz_cache
    raw_input = coords_widget.value.strip()
    if not raw_input:
        with mol_output:
            clear_output()
            print("Please enter SMILES in the input box.")
        return

    with mol_output:
        clear_output()
        print("Converting SMILES...")

    cleaned_data, input_type = clean_input_data(raw_input)
    if input_type != 'smiles':
        with mol_output:
            clear_output()
            print("Input is not a SMILES string.")
        return

    xyz_string, num_atoms, method, error = smiles_to_xyz(cleaned_data)
    if error:
        with mol_output:
            clear_output()
            print(f"SMILES: {cleaned_data}")
            print(f"Fehler: {error}")
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        return

    _converted_xyz_cache = {'smiles': cleaned_data, 'xyz': xyz_string}
    coords_widget.value = f"{num_atoms}\nConverted from SMILES ({method})\n{xyz_string}"


convert_smiles_button.on_click(handle_convert_smiles)


def handle_build_complex(button):
    """Submit a build-complex job."""
    with output_area:
        clear_output()

        job_name = job_name_widget.value.strip()
        if not job_name:
            print("Error: Job name is required!")
            return

        raw_input = coords_widget.value.strip()
        if not raw_input:
            print("Error: Please enter a SMILES string in the input box.")
            return

        cleaned_data, input_type = clean_input_data(raw_input)
        if input_type != 'smiles':
            print("Error: Input must be a SMILES string for BUILD COMPLEX.")
            return

        if not contains_metal(cleaned_data):
            print("Error: SMILES does not contain a metal atom.")
            return

        safe_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        job_dir = BASE_DIR / safe_name

        if job_dir.exists():
            print(f"Error: Directory already exists: {job_dir}")
            return

        try:
            job_dir.mkdir(parents=True)
        except Exception as e:
            print(f"Error creating directory: {e}")
            return

        input_file = job_dir / 'input.txt'
        input_file.write_text(cleaned_data + '\n')

        time_limit = _resolve_time_limit(job_type_widget, custom_time_widget, '48:00:00')

        try:
            result = local_submit(
                job_dir=job_dir,
                mode='build',
                job_name=safe_name,
                time_limit=time_limit,
                build_mult=1,
                pal=32,
                maxcore=4000
            )

            if result.returncode == 0:
                print(f"Job submitted successfully!")
                print(result.stdout)
                print(f"")
                print(f"Check status in Job Status tab")
                reset_form()
            else:
                print(f"Submission failed:")
                print(result.stderr)
        except Exception as e:
            print(f"Error submitting job: {e}")


build_complex_button.on_click(handle_build_complex)


def handle_submit_smiles_list(button):
    with smiles_batch_output:
        clear_output()

        job_prefix = job_name_widget.value.strip()
        if not job_prefix:
            print("Error: Job name cannot be empty!")
            return

        control_content_base = control_widget.value
        control_errors = validate_control_text(control_content_base)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        time_limit = _resolve_time_limit(job_type_widget, custom_time_widget, '48:00:00')

        entries = [l.strip() for l in smiles_batch_widget.value.splitlines() if l.strip()]
        if not entries:
            print("Error: SMILES list is empty.")
            return

        for idx, entry in enumerate(entries, 1):
            if ';' not in entry:
                print(f"Line {idx}: Missing ';' delimiter -> {entry}")
                continue
            parts = [p.strip() for p in entry.split(';') if p.strip()]
            if len(parts) < 2:
                print(f"Line {idx}: Missing name or SMILES -> {entry}")
                continue

            name_raw = parts[0]
            smiles = parts[1]
            extra_parts = parts[2:]

            if not name_raw or not smiles:
                print(f"Line {idx}: Missing name or SMILES -> {entry}")
                continue

            extras = {}
            for part in extra_parts:
                if '=' not in part:
                    print(f"Line {idx}: Invalid override '{part}' (expected key=value)")
                    continue
                key, value = part.split('=', 1)
                key = key.strip()
                value = value.strip()
                if not key:
                    print(f"Line {idx}: Invalid override '{part}' (empty key)")
                    continue
                extras[key] = value

            safe_name = ''.join(c for c in name_raw if c.isalnum() or c in ('_', '-'))
            if not safe_name:
                print(f"Line {idx}: Invalid name -> {name_raw}")
                continue

            job_name = f"{job_prefix}_{safe_name}"
            safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
            if not safe_job_name:
                print(f"Line {idx}: Invalid job name -> {job_name}")
                continue

            xyz_string, num_atoms, method, error = smiles_to_xyz(smiles)
            if error:
                print(f"Line {idx}: {safe_name} - SMILES error: {error}")
                continue

            job_dir = BASE_DIR / safe_job_name
            job_dir.mkdir(parents=True, exist_ok=True)

            control_content = re.sub(r'(?m)^SMILES=.*$', f"SMILES={smiles}", control_content_base)
            for key, value in extras.items():
                pattern = rf'(?m)^{re.escape(key)}\s*=.*$'
                replacement = f"{key}={value}"
                if re.search(pattern, control_content):
                    control_content = re.sub(pattern, replacement, control_content)
                else:
                    control_content = control_content.rstrip() + f"\n{replacement}\n"

            control_path = job_dir / 'CONTROL.txt'
            control_path.write_text(control_content)

            input_path = job_dir / 'input.txt'
            input_path.write_text(xyz_string)

            pal, maxcore = parse_resource_settings(control_content)
            total_mem = pal * maxcore if pal and maxcore else 240000

            result = submit_job(
                job_dir=job_dir,
                mode='delfin',
                job_name=safe_job_name,
                time_limit=time_limit,
                ntasks=pal or 40,
                mem_mb=total_mem
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
                print(f"Submitted {safe_job_name} (ID: {job_id})")
            else:
                print(f"Failed {safe_job_name}: {result.stderr or result.stdout}")


submit_smiles_list_button.on_click(handle_submit_smiles_list)


def get_smiles_list_entries():
    """Parse SMILES batch widget and return list of (name, smiles, extras) tuples."""
    entries = []
    for line in smiles_batch_widget.value.splitlines():
        line = line.strip()
        if not line or ';' not in line:
            continue
        parts = [p.strip() for p in line.split(';') if p.strip()]
        if len(parts) >= 2:
            name = parts[0]
            smiles = parts[1]
            extras = {}
            for part in parts[2:]:
                if '=' in part:
                    k, v = part.split('=', 1)
                    extras[k.strip()] = v.strip()
            entries.append((name, smiles, extras))
    return entries


def update_smiles_preview_label():
    entries = get_smiles_list_entries()
    total = len(entries)
    if total == 0:
        smiles_preview_label.value = '<span style="font-size:12px;">0 / 0</span>'
    else:
        current = _smiles_preview_index['current'] + 1
        smiles_preview_label.value = f'<span style="font-size:12px;">{current} / {total}</span>'


def preview_smiles_at_index(index):
    entries = get_smiles_list_entries()
    if not entries:
        with mol_output:
            clear_output()
            print("No valid SMILES entries in the batch list.")
        return

    if index < 0:
        index = 0
    if index >= len(entries):
        index = len(entries) - 1

    _smiles_preview_index['current'] = index
    update_smiles_preview_label()

    name, smiles, extras = entries[index]

    with mol_output:
        clear_output()
        print(f"Preview: {name}")
        print(f"SMILES: {smiles}")
        if extras:
            print(f"Options: {extras}")
        print("Converting...")

    xyz_string, num_atoms, method, error = smiles_to_xyz(smiles)

    with mol_output:
        clear_output()
        if error:
            print(f"Preview: {name}")
            print(f"SMILES: {smiles}")
            if extras:
                print(f"Options: {extras}")
            print(f"Error: {error}")
            return

        print(f"Preview: {name} ({method})")
        print(f"SMILES: {smiles}")
        if extras:
            print(f"Options: {extras}")

        xyz_data = f"{num_atoms}\nPreview: {name}\n{xyz_string}"
        mol_view = py3Dmol.view(width=560, height=350)
        mol_view.addModel(xyz_data, "xyz")
        mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
        mol_view.zoomTo()
        mol_view.show()


def handle_smiles_prev(button):
    entries = get_smiles_list_entries()
    if not entries:
        with mol_output:
            clear_output()
            print("No valid SMILES entries in the batch list.")
        return
    new_index = _smiles_preview_index['current'] - 1
    if new_index < 0:
        new_index = len(entries) - 1
    preview_smiles_at_index(new_index)


def handle_smiles_next(button):
    entries = get_smiles_list_entries()
    if not entries:
        with mol_output:
            clear_output()
            print("No valid SMILES entries in the batch list.")
        return
    new_index = _smiles_preview_index['current'] + 1
    if new_index >= len(entries):
        new_index = 0
    preview_smiles_at_index(new_index)


smiles_prev_button.on_click(handle_smiles_prev)
smiles_next_button.on_click(handle_smiles_next)


def reset_form():
    """Reset form after successful submit."""
    global _converted_xyz_cache
    job_name_widget.value = ''
    coords_widget.value = ''
    smiles_batch_widget.value = ''
    control_widget.value = DEFAULT_CONTROL
    job_type_widget.value = '48h'
    custom_time_widget.value = 72
    only_goat_charge.value = 0
    only_goat_solvent.value = ''
    _converted_xyz_cache = {'smiles': None, 'xyz': None}
    with mol_output:
        clear_output()
        print("Please enter XYZ coordinates or SMILES.")


def handle_submit(button):
    with output_area:
        clear_output()

        job_name = job_name_widget.value.strip()
        control_content = control_widget.value
        raw_input = coords_widget.value.strip()

        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return

        control_errors = validate_control_text(control_content)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        input_content, input_type = clean_input_data(raw_input)

        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'

        if not input_content:
            print("Error: No valid input found!")
            return

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        job_dir = BASE_DIR / safe_job_name
        time_limit = _resolve_time_limit(job_type_widget, custom_time_widget, '48:00:00')

        try:
            job_dir.mkdir(parents=True, exist_ok=True)

            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace(
                    'SMILES=',
                    f"SMILES={_converted_xyz_cache['smiles']}"
                )

            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)

            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)

            pal, maxcore = parse_resource_settings(control_content)
            total_mem = pal * maxcore if pal and maxcore else 240000

            result = submit_job(
                job_dir=job_dir,
                mode='delfin',
                job_name=safe_job_name,
                time_limit=time_limit,
                ntasks=pal or 40,
                mem_mb=total_mem
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f"Job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Time Limit: {time_limit}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Check status in Job Status tab")
                reset_form()
            else:
                print(f"Error submitting job:")
                print(result.stderr)

        except Exception as e:
            print(f"Error creating job: {str(e)}")


def handle_validate_control(button):
    with validate_output:
        clear_output()
        control_content = control_widget.value
        control_errors = validate_control_text(control_content)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
        else:
            print("CONTROL.txt looks valid.")


def handle_only_goat_submit(button):
    with only_goat_output:
        clear_output()

        job_name = job_name_widget.value.strip()
        raw_input = coords_widget.value.strip()
        charge_value = only_goat_charge.value
        solvent_value = only_goat_solvent.value.strip()

        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return

        if solvent_value == '':
            print("Error: Solvent cannot be empty!")
            return

        input_content, input_type = clean_input_data(raw_input)

        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'

        if not input_content:
            print("Error: No valid input found!")
            return

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        job_dir = BASE_DIR / safe_job_name
        time_limit = _resolve_time_limit(job_type_widget, custom_time_widget, '48:00:00')

        try:
            job_dir.mkdir(parents=True, exist_ok=True)

            control_content = (ONLY_GOAT_TEMPLATE
                .replace('[CHARGE]', str(charge_value))
                .replace('[SOLVENT]', solvent_value)
            )

            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace(
                    'SMILES=',
                    f"SMILES={_converted_xyz_cache['smiles']}"
                )

            control_errors = validate_control_text(control_content)
            if control_errors:
                print("CONTROL.txt validation failed:")
                for err in control_errors:
                    print(f"- {err}")
                return
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)

            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)

            pal, maxcore = parse_resource_settings(control_content)
            total_mem = pal * maxcore if pal and maxcore else 240000

            result = submit_job(
                job_dir=job_dir,
                mode='delfin',
                job_name=safe_job_name,
                time_limit=time_limit,
                ntasks=pal or 40,
                mem_mb=total_mem
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
                print("Only GOAT job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Time Limit: {time_limit}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Charge: {charge_value}")
                print(f"Solvent: {solvent_value}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Check status in Job Status tab")
                reset_form()
            else:
                print("Error submitting job:")
                print(result.stderr)

        except Exception as e:
            print(f"Error creating job: {str(e)}")


only_goat_submit_button.on_click(handle_only_goat_submit)
validate_button.on_click(handle_validate_control)
submit_button.on_click(handle_submit)

spacer = widgets.Label(value="", layout=widgets.Layout(height='20px'))

ui_left = widgets.VBox([
    job_name_widget,
    spacer,
    job_type_widget,
    custom_time_widget,
    spacer,
    coords_help,
    coords_widget,
    button_spacer,
    widgets.HBox([convert_smiles_button, build_complex_button], layout=widgets.Layout(gap='10px')),
    spacer,
    smiles_batch_help,
    smiles_batch_widget,
    button_spacer,
    widgets.HBox([smiles_prev_button, smiles_preview_label, smiles_next_button, submit_smiles_list_button], layout=widgets.Layout(gap='2px', align_items='center')),
    smiles_batch_output,
    spacer,
    control_help,
    control_widget,
    button_spacer,
    validate_button,
    submit_button,
    output_area,
    validate_output
], layout=widgets.Layout(width='55%', padding='10px'))

ui_right = widgets.VBox([
    widgets.Label("Molecule visualization:"),
    mol_output,
    xyz_copy_row,
    spacer,
    only_goat_label,
    widgets.HBox([
        only_goat_charge,
        only_goat_solvent,
        only_goat_submit_button
    ], layout=widgets.Layout(width='600px')),
    only_goat_output
], layout=widgets.Layout(width='45%', padding='10px'))

full_ui = widgets.HBox([ui_left, ui_right])
tab1_content = full_ui

# === DELFIN RECALC MODULES ===
recalc_title = widgets.HTML("<h3>DELFIN RECALC MODULES</h3>")
recalc_help = widgets.Label("Select a job folder from calc, edit CONTROL.txt, then submit a recalc job.")

recalc_folder_dropdown = widgets.Dropdown(
    options=[],
    description='Job Folder:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': 'initial'}
)

recalc_time_limit = widgets.ToggleButtons(
    options=['12h', '24h', '36h', '48h', '60h', 'Custom'],
    value='24h',
    description='Time Limit:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='550px')
)

recalc_custom_time_widget = widgets.BoundedIntText(
    value=72,
    min=1,
    max=720,
    step=1,
    description='Hours:',
    layout=widgets.Layout(width='180px', display='none'),
    style={'description_width': 'initial'}
)

def _toggle_recalc_custom_time_visibility(change):
    if change['new'] == 'Custom':
        recalc_custom_time_widget.layout.display = ''
    else:
        recalc_custom_time_widget.layout.display = 'none'

recalc_time_limit.observe(_toggle_recalc_custom_time_visibility, names='value')

recalc_control_widget = widgets.Textarea(
    value='',
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='400px'),
    style={'description_width': 'initial'}
)

recalc_spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))

recalc_button = widgets.Button(
    description='SUBMIT RECALC',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)
recalc_output = widgets.Output()


def refresh_recalc_folders():
    if not BASE_DIR.exists():
        recalc_folder_dropdown.options = []
        recalc_folder_dropdown.value = None
        return

    folders = sorted([p.name for p in BASE_DIR.iterdir() if p.is_dir()])
    recalc_folder_dropdown.options = folders
    recalc_folder_dropdown.value = None


def load_recalc_control(change=None):
    with recalc_output:
        clear_output()

        folder = recalc_folder_dropdown.value
        if not folder:
            recalc_control_widget.value = ''
            return

        control_path = BASE_DIR / folder / 'CONTROL.txt'
        if control_path.exists():
            recalc_control_widget.value = control_path.read_text()
        else:
            recalc_control_widget.value = ''
            print(f"CONTROL.txt not found in {control_path.parent}")


def handle_recalc(button):
    with recalc_output:
        clear_output()

        folder = recalc_folder_dropdown.value
        if not folder:
            print("Error: Please select a job folder.")
            return

        job_dir = BASE_DIR / folder
        if not job_dir.exists():
            print(f"Error: Job folder does not exist: {job_dir}")
            return

        control_errors = validate_control_text(recalc_control_widget.value)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        control_path = job_dir / 'CONTROL.txt'
        control_path.write_text(recalc_control_widget.value)

        pal, maxcore = parse_resource_settings(recalc_control_widget.value)
        if pal is None or maxcore is None:
            print('Error: PAL/maxcore not found in CONTROL.txt')
            return
        total_mem = pal * maxcore

        time_limit = _resolve_time_limit(recalc_time_limit, recalc_custom_time_widget, '24:00:00')

        result = submit_job(
            job_dir=job_dir,
            mode='delfin-recalc',
            job_name=job_dir.name,
            time_limit=time_limit,
            ntasks=pal,
            mem_mb=total_mem
        )

        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("Recalc job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Directory: {job_dir}")
        else:
            print("Error submitting recalc job:")
            print(result.stderr or result.stdout)


recalc_folder_dropdown.observe(load_recalc_control, names='value')
refresh_recalc_folders()

recalc_section = widgets.VBox([
    recalc_title,
    recalc_help,
    recalc_folder_dropdown,
    recalc_time_limit,
    recalc_custom_time_widget,
    recalc_control_widget,
    recalc_spacer,
    recalc_button,
    recalc_output
], layout=widgets.Layout(width='600px', padding='10px'))

recalc_button.on_click(handle_recalc)

tab2_content = recalc_section

# === Job Status (Local - replaces squeue/scancel) ===
job_status_title = widgets.HTML("<h3>Job Status</h3>")
job_status_help = widgets.HTML("<p>Select a job and click CANCEL to cancel it. Click REFRESH to update.</p>")

_job_data = []

job_table_html = widgets.HTML(value="<i>Loading...</i>")
job_start_html = widgets.HTML(value="")

job_dropdown = widgets.Dropdown(
    options=[],
    description='Select Job:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '80px'}
)

job_status_output = widgets.Output()

def refresh_job_list(button=None):
    """Refresh the job list from local JSON registry."""
    global _job_data

    with job_status_output:
        clear_output()

    try:
        data = _load_jobs()
        jobs = data.get('jobs', [])

        # Update status of all running jobs
        for job in jobs:
            _update_job_status(job)
        _save_jobs(data)

        # Filter: only show active jobs (RUNNING + PENDING)
        visible_jobs = [job for job in jobs if job['status'] in ('RUNNING', 'PENDING')]

        if not visible_jobs:
            job_table_html.value = "<p><i>No jobs found.</i></p>"
            job_start_html.value = ""
            job_dropdown.options = []
            _job_data = []
            with job_status_output:
                print("0 jobs found")
            return

        _job_data = visible_jobs
        dropdown_options = []

        # Status colors
        status_colors = {
            'PENDING': '#9E9E9E',
            'RUNNING': '#4CAF50',
            'COMPLETED': '#2196F3',
            'FAILED': '#f44336',
            'CANCELLED': '#FF9800',
            'TIMEOUT': '#b71c1c',
        }

        table_html = """
        <style>
            .job-table { font-family: monospace; font-size: 12px; border-collapse: collapse; width: 100%; }
            .job-table th, .job-table td { padding: 6px 10px; text-align: left; border-bottom: 1px solid #ddd; }
            .job-table th { background-color: #2196F3; color: white; }
            .job-table tr:hover { background-color: #f5f5f5; }
        </style>
        <table class="job-table">
            <tr>
                <th>JOB ID</th>
                <th>NAME</th>
                <th>MODE</th>
                <th>STATUS</th>
                <th>QUEUE</th>
                <th>SUBMITTED</th>
                <th>PAL</th>
                <th>TIME LIMIT</th>
                <th>DIRECTORY</th>
            </tr>
        """

        # Calculate queue positions for PENDING jobs
        pending_jobs_sorted = [j for j in sorted(visible_jobs, key=lambda j: j.get('job_id', 0)) if j['status'] == 'PENDING']
        queue_positions = {j.get('job_id'): idx + 1 for idx, j in enumerate(pending_jobs_sorted)}

        for job in sorted(visible_jobs, key=lambda j: j.get('job_id', 0), reverse=True):
            job_id = job.get('job_id', '?')
            name = job.get('name', '?')
            mode = job.get('mode', '?')
            status = job.get('status', '?')
            submit_time = job.get('submit_time', '?')
            pal = job.get('pal', '?')
            time_limit = job.get('time_limit', '?')
            job_dir = job.get('job_dir', '?')

            # Format submit time
            try:
                dt = datetime.fromisoformat(submit_time)
                submit_display = dt.strftime('%Y-%m-%d %H:%M')
            except:
                submit_display = submit_time

            # Short directory
            try:
                dir_display = str(Path(job_dir).name)
            except:
                dir_display = job_dir

            color = status_colors.get(status, '#333')

            table_html += f"""
            <tr>
                <td><b>{job_id}</b></td>
                <td>{html.escape(str(name))}</td>
                <td>{html.escape(str(mode))}</td>
                <td><b style="color:{color};">{status}</b></td>
                <td>{queue_positions.get(job_id, '-')}</td>
                <td>{submit_display}</td>
                <td>{pal}</td>
                <td>{time_limit}</td>
                <td title="{html.escape(str(job_dir))}">{html.escape(dir_display)}</td>
            </tr>
            """

            if status in ('RUNNING', 'PENDING'):
                dropdown_options.append((f"{job_id} - {name}", job_id))

        table_html += "</table>"
        job_table_html.value = table_html
        job_start_html.value = ""

        if dropdown_options:
            job_dropdown.options = dropdown_options
            job_dropdown.value = None
        else:
            job_dropdown.options = []

        with job_status_output:
            clear_output()
            running = sum(1 for j in visible_jobs if j['status'] == 'RUNNING')
            pending = sum(1 for j in visible_jobs if j['status'] == 'PENDING')
            parts = [f"{len(visible_jobs)} job(s) shown"]
            if running:
                parts.append(f"{running} running")
            if pending:
                parts.append(f"{pending} in queue")
            print(", ".join(parts))

    except Exception as e:
        with job_status_output:
            clear_output()
            print(f"Error loading jobs: {e}")

def cancel_selected_job(button):
    """Cancel the currently selected job."""
    with job_status_output:
        clear_output()

        if not job_dropdown.value:
            print("No job selected.")
            return

        job_id = job_dropdown.value
        success, msg = _cancel_job(job_id)
        print(msg)
        if success:
            refresh_job_list()

# Buttons
refresh_button = widgets.Button(
    description='REFRESH',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

cancel_button = widgets.Button(
    description='CANCEL JOB',
    button_style='danger',
    layout=widgets.Layout(width='150px')
)

refresh_button.on_click(refresh_job_list)
cancel_button.on_click(cancel_selected_job)

# Initial load
refresh_job_list()

# Layout
button_row = widgets.HBox([
    job_dropdown,
    cancel_button,
    refresh_button
], layout=widgets.Layout(margin='10px 0', align_items='center'))

job_status_section = widgets.VBox([
    job_status_title,
    job_status_help,
    job_table_html,
    job_start_html,
    button_row,
    job_status_output
], layout=widgets.Layout(width='600px', padding='10px', border='1px solid #2196F3'))

tab3_content = job_status_section

# === ORCA Input Builder ===
from delfin.common.control_validator import ORCA_FUNCTIONALS, ORCA_BASIS_SETS, DISP_CORR_VALUES, _RI_JKX_KEYWORDS

orca_method_options = sorted(ORCA_FUNCTIONALS)
orca_basis_options = sorted(ORCA_BASIS_SETS)
orca_dispersion_options = ['None'] + sorted(v for v in DISP_CORR_VALUES if v)
orca_ri_options = ['None'] + sorted(_RI_JKX_KEYWORDS)
orca_aux_basis_options = ['None', 'def2/J', 'def2/JK']

orca_builder_title = widgets.HTML("<h3>ORCA Input Builder</h3>")
orca_builder_help = widgets.Label("Build ORCA input files and submit standalone ORCA calculations.")

# === ORCA Builder Widgets ===
orca_job_name = widgets.Text(
    value='',
    placeholder='e.g. water_opt',
    description='Job Name:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': '120px'}
)

orca_coords = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates (with or without header):\n\n6\nComment line\nC  0.0  0.0  0.0\n...\n\nor just:\nC  0.0  0.0  0.0\n...:\nO   0.000000   0.000000   0.117300\nH   0.000000   0.756950  -0.469200\nH   0.000000  -0.756950  -0.469200',
    description='Coordinates:',
    layout=widgets.Layout(width='100%', height='400px'),
    style={'description_width': '120px'}
)

orca_charge = widgets.IntText(
    value=0,
    description='Charge:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_multiplicity = widgets.IntText(
    value=1,
    description='Multiplicity:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_method = widgets.Dropdown(
    options=orca_method_options,
    value='PBE0',
    description='Method:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_job_type = widgets.Dropdown(
    options=['SP', 'OPT', 'FREQ', 'OPT FREQ'],
    value='OPT',
    description='Job Type:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_basis = widgets.Dropdown(
    options=orca_basis_options,
    value='def2-SVP',
    description='Basis Set:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_dispersion = widgets.Dropdown(
    options=orca_dispersion_options,
    value='D4',
    description='Dispersion:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_ri = widgets.Dropdown(
    options=orca_ri_options,
    value='RIJCOSX',
    description='RI Approx:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_aux_basis = widgets.Dropdown(
    options=orca_aux_basis_options,
    value='def2/J',
    description='Aux Basis:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_autoaux = widgets.Checkbox(
    value=False,
    description='AutoAux',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

# === Solvation ===
orca_solvation_type = widgets.Dropdown(
    options=['None', 'CPCM', 'SMD'],
    value='None',
    description='Solvation Type:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_solvent = widgets.Dropdown(
    options=['water', 'acetonitrile', 'dmso', 'dmf', 'methanol', 'ethanol', 'thf', 'dichloromethane', 'chloroform', 'toluene', 'hexane'],
    value='water',
    description='Solvent:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

# === Print options ===
orca_print_mos = widgets.Checkbox(
    value=False,
    description='Print MOs',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_print_basis = widgets.Checkbox(
    value=False,
    description='Print Basis',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_additional = widgets.Text(
    value='',
    placeholder='e.g. FinalGrid6 NormalPrint',
    description='Additional:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '120px'}
)

orca_pal = widgets.IntText(
    value=40,
    description='PAL (cores):',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_maxcore = widgets.IntText(
    value=6000,
    description='MaxCore (MB):',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_slurm_time = widgets.Text(
    value='12:00:00',
    placeholder='e.g. 02:00:00',
    description='Time Limit:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

# === File Upload ===
orca_file_upload = widgets.FileUpload(
    accept='',
    multiple=True,
    description='Extra Files:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '120px'}
)

orca_uploaded_files_label = widgets.HTML(
    value='<i>Drag & drop files here (e.g. .gbw, .xyz, .hess)</i>',
    layout=widgets.Layout(width='100%')
)

orca_path_files = widgets.Textarea(
    value='',
    placeholder='Paste file paths (one per line):\n/path/to/file.gbw\n/path/to/file.xyz',
    description='File Paths:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '120px'}
)

# Preview area
orca_preview = widgets.Textarea(
    value='',
    description='INP Preview:',
    layout=widgets.Layout(width='600px', height='550px'),
    style={'description_width': '120px'},
    disabled=False
)

# Buttons
orca_generate_btn = widgets.Button(
    description='GENERATE INP',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

orca_save_btn = widgets.Button(
    description='SAVE INP + SH',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

orca_submit_btn = widgets.Button(
    description='SUBMIT ORCA JOB',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

orca_output = widgets.Output()

_orca_extra_files = {}

orca_mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #4CAF50',
    width='100%',
    min_height='300px',
    overflow='hidden'
))


def reset_orca_builder():
    """Reset all ORCA Builder fields to their default values."""
    orca_job_name.value = ''
    orca_coords.value = ''
    orca_charge.value = 0
    orca_multiplicity.value = 1
    orca_method.value = 'PBE0'
    orca_job_type.value = 'OPT'
    orca_basis.value = 'def2-SVP'
    orca_dispersion.value = 'D4'
    orca_ri.value = 'RIJCOSX'
    orca_aux_basis.value = 'def2/J'
    orca_solvation_type.value = 'None'
    orca_solvent.value = 'water'
    orca_print_mos.value = False
    orca_print_basis.value = False
    orca_additional.value = ''
    orca_pal.value = 40
    orca_maxcore.value = 6000
    orca_slurm_time.value = '12:00:00'
    orca_file_upload.value = ()
    orca_uploaded_files_label.value = ''
    orca_path_files.value = ''
    _orca_extra_files.clear()
    update_orca_preview()


def strip_xyz_header(text):
    """Remove XYZ header (atom count + comment line) if present."""
    text = text.strip()
    if not text:
        return text

    lines = text.split('\n')
    if len(lines) < 2:
        return text

    first_line = lines[0].strip()
    try:
        int(first_line)
        return '\n'.join(lines[2:]).strip()
    except ValueError:
        return text


def update_orca_molecule_view(change=None):
    """Update molecule visualization for ORCA Builder."""
    with orca_mol_output:
        clear_output()
        raw_input = orca_coords.value.strip()

        if not raw_input:
            print("Paste XYZ coordinates to see 3D preview.")
            return

        coords = strip_xyz_header(raw_input)

        if not coords:
            print("No valid coordinates.")
            return

        try:
            lines = [l for l in coords.split('\n') if l.strip()]
            num_atoms = len(lines)
            xyz_data = f"{num_atoms}\nORCA Builder Preview\n{coords}"
            mol_view = py3Dmol.view(width='100%', height=400)
            mol_view.addModel(xyz_data, "xyz")
            mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
            mol_view.zoomTo()
            mol_view.show()
        except Exception as e:
            print(f"Could not visualize: {e}")


def generate_orca_input():
    """Generate ORCA input file content from widget values."""
    keywords = []
    keywords.append(orca_method.value)
    keywords.append(orca_job_type.value)
    keywords.append(orca_basis.value)

    if orca_dispersion.value != 'None':
        keywords.append(orca_dispersion.value)
    if orca_ri.value != 'None':
        keywords.append(orca_ri.value)
    if orca_ri.value != 'None':
        keywords.append(orca_aux_basis.value)
        if orca_autoaux.value:
            keywords.append('AutoAux')
    if orca_solvation_type.value != 'None':
        keywords.append(f"{orca_solvation_type.value}({orca_solvent.value})")

    output_lines = []
    if orca_print_mos.value:
        output_lines.append("  print[p_mos] 1")
    if orca_print_basis.value:
        output_lines.append("  print[p_basis] 2")
    output_block = ""
    if output_lines:
        output_block = "%output\n" + "\n".join(output_lines) + "\nend"

    if orca_additional.value.strip():
        keywords.append(orca_additional.value.strip())

    keyword_line = '! ' + ' '.join(keywords)

    pal_block = f"""%pal
  nprocs {orca_pal.value}
end"""

    maxcore_line = f"%maxcore {orca_maxcore.value}"

    coords = strip_xyz_header(orca_coords.value)
    coord_block = f"""* xyz {orca_charge.value} {orca_multiplicity.value}
{coords}
*"""

    inp_content = f"""{keyword_line}

{pal_block}

{maxcore_line}
"""
    if output_block:
        inp_content += f"\n{output_block}\n"
    inp_content += f"\n{coord_block}\n"
    return inp_content

def parse_inp_resources(inp_text):
    """Parse PAL (nprocs) and maxcore from ORCA input text."""
    pal = None
    maxcore = None
    if not inp_text:
        return pal, maxcore
    pal_match = re.search(r'(?im)^\s*nprocs\s+(\d+)', inp_text)
    if pal_match:
        pal = int(pal_match.group(1))
    maxcore_match = re.search(r'(?im)^\s*%maxcore\s+(\d+)', inp_text)
    if maxcore_match:
        maxcore = int(maxcore_match.group(1))
    return pal, maxcore


def save_uploaded_files(job_dir):
    """Save uploaded files (drag & drop) and copy path-based files to job directory."""
    saved_files = []
    files_to_save = _orca_extra_files if _orca_extra_files else {}
    if not files_to_save and orca_file_upload.value:
        for uploaded_file in orca_file_upload.value:
            filename = uploaded_file['name'] if isinstance(uploaded_file, dict) else uploaded_file.name
            content = uploaded_file['content'] if isinstance(uploaded_file, dict) else uploaded_file.content
            files_to_save[filename] = content
    for filename, content in files_to_save.items():
        file_path = job_dir / filename
        file_path.write_bytes(content)
        saved_files.append(filename)
    path_lines = orca_path_files.value.strip().splitlines()
    for line in path_lines:
        p = Path(line.strip())
        if not p.name:
            continue
        if not p.exists():
            print(f"Warning: File not found, skipped: {p}")
            continue
        if p.is_file():
            dest = job_dir / p.name
            shutil.copy2(str(p), str(dest))
            saved_files.append(p.name)
        elif p.is_dir():
            print(f"Warning: '{p.name}' is a directory, skipped.")
    return saved_files


def update_uploaded_files_label(change=None):
    if orca_file_upload.value:
        for f in orca_file_upload.value:
            name = f['name'] if isinstance(f, dict) else f.name
            content = f['content'] if isinstance(f, dict) else f.content
            _orca_extra_files[name] = content
    if _orca_extra_files:
        filenames = list(_orca_extra_files.keys())
        orca_uploaded_files_label.value = f"<b>Files to upload:</b> {', '.join(filenames)}"
    else:
        orca_uploaded_files_label.value = '<i>Drag & drop files here (e.g. .gbw, .xyz, .hess)</i>'


orca_file_upload.observe(update_uploaded_files_label, names='value')


def sanitize_orca_input(text: str) -> str:
    """Sanitize ORCA input to avoid hidden/invalid characters."""
    if text is None:
        return ''
    text = text.replace("\r\n", "\n").replace("\r", "\n").lstrip("\ufeff")
    text = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]", "", text)
    text = ''.join(ch for ch in text if ch == "\n" or ch == "\t" or (" " <= ch <= "~"))
    lines = text.split("\n")
    out_lines = []
    for line in lines:
        if re.search(r"^\s*\*\s*xyzfile\b", line, flags=re.IGNORECASE):
            parts = line.split()
            if len(parts) >= 5:
                filename = parts[4]
                filename = filename.strip("\"'")
                filename = re.sub(r"[^A-Za-z0-9._/+-]", "", filename)
                m = re.match(r"(.+?\.xyz)", filename, flags=re.IGNORECASE)
                if m:
                    filename = m.group(1)
                parts = parts[:4] + [filename] + parts[5:]
                line = ' '.join(parts)
        out_lines.append(line)
    return "\n".join(out_lines).strip() + "\n"

def _build_keyword_line():
    """Build the ! keyword line from widget values."""
    keywords = []
    keywords.append(orca_method.value)
    keywords.append(orca_job_type.value)
    keywords.append(orca_basis.value)
    if orca_dispersion.value != 'None':
        keywords.append(orca_dispersion.value)
    if orca_ri.value != 'None':
        keywords.append(orca_ri.value)
        keywords.append(orca_aux_basis.value)
        if orca_autoaux.value:
            keywords.append('AutoAux')
    if orca_solvation_type.value != 'None':
        keywords.append(f"{orca_solvation_type.value}({orca_solvent.value})")
    if orca_additional.value.strip():
        keywords.append(orca_additional.value.strip())
    return '! ' + ' '.join(keywords)


_last_auto_keywords = ''


def _build_output_block():
    """Build the %output block from widget values."""
    output_lines = []
    if orca_print_mos.value:
        output_lines.append("  print[p_mos] 1")
    if orca_print_basis.value:
        output_lines.append("  print[p_basis] 2")
    if output_lines:
        return "%output\n" + "\n".join(output_lines) + "\nend"
    return ""


def update_orca_preview(change=None):
    """Incrementally update affected parts of the ORCA input preview."""
    global _last_auto_keywords
    current = orca_preview.value.strip()

    if not current:
        orca_preview.value = sanitize_orca_input(generate_orca_input())
        _last_auto_keywords = _build_keyword_line()
        return

    text = current

    new_kw = _build_keyword_line()
    kw_match = re.search(r'^(!.*)$', text, re.MULTILINE)
    if kw_match:
        old_line = kw_match.group(1)
        manual_suffix = ''
        if _last_auto_keywords and old_line.startswith(_last_auto_keywords):
            manual_suffix = old_line[len(_last_auto_keywords):]
        text = re.sub(r'^!.*$', new_kw + manual_suffix, text, count=1, flags=re.MULTILINE)
    _last_auto_keywords = new_kw

    new_pal = f"%pal\n  nprocs {orca_pal.value}\nend"
    text = re.sub(
        r'%pal\s*\n\s*nprocs\s+\d+\s*\n\s*end',
        new_pal, text, count=1, flags=re.IGNORECASE
    )

    text = re.sub(
        r'^%maxcore\s+\d+',
        f'%maxcore {orca_maxcore.value}',
        text, count=1, flags=re.MULTILINE | re.IGNORECASE
    )

    new_output = _build_output_block()
    has_output = re.search(r'%output\b.*?\nend', text, flags=re.DOTALL | re.IGNORECASE)
    if has_output:
        if new_output:
            text = re.sub(
                r'%output\b.*?\nend',
                new_output, text, count=1, flags=re.DOTALL | re.IGNORECASE
            )
        else:
            text = re.sub(
                r'\n?%output\b.*?\nend\n?',
                '\n', text, count=1, flags=re.DOTALL | re.IGNORECASE
            )
    elif new_output:
        text = re.sub(
            r'(\n\* xyz )',
            f'\n{new_output}\n\\1', text, count=1
        )

    coords = strip_xyz_header(orca_coords.value)
    new_coord = f"* xyz {orca_charge.value} {orca_multiplicity.value}\n{coords}\n*"
    text = re.sub(
        r'\*\s*xyz\s+[-\d]+\s+\d+\s*\n.*?\n\*',
        new_coord, text, count=1, flags=re.DOTALL
    )

    orca_preview.value = sanitize_orca_input(text)

def handle_orca_generate(button):
    """Handle GENERATE INP button click."""
    global _last_auto_keywords
    orca_preview.value = sanitize_orca_input(generate_orca_input())
    _last_auto_keywords = _build_keyword_line()
    with orca_output:
        clear_output()
        print("ORCA input regenerated. You can edit the preview if needed.")


def handle_orca_save(button):
    """Handle SAVE INP + SH button click."""
    with orca_output:
        clear_output()

        job_name = orca_job_name.value.strip()
        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)

        preview_content = orca_preview.value.strip()
        if preview_content:
            inp_content = preview_content
        else:
            coords = strip_xyz_header(orca_coords.value)
            if not coords:
                print("Error: Coordinates or INP preview cannot be empty!")
                return
            inp_content = generate_orca_input()

        inp_content = sanitize_orca_input(inp_content)

        inp_path = job_dir / f"{safe_job_name}.inp"
        inp_path.write_text(inp_content)

        saved_files = save_uploaded_files(job_dir)

        pal_used, maxcore_used = parse_inp_resources(inp_content)
        if pal_used is None:
            pal_used = orca_pal.value
        if maxcore_used is None:
            maxcore_used = orca_maxcore.value
        total_mem = pal_used * maxcore_used

        result = submit_job(
            job_dir=job_dir,
            mode='orca',
            job_name=safe_job_name,
            inp_file=f'{safe_job_name}.inp',
            time_limit=orca_slurm_time.value,
            ntasks=pal_used,
            mem_mb=total_mem
        )

        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("ORCA job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Job Name: {safe_job_name}")
            print(f"Directory: {job_dir}")
            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            print(f"PAL: {pal_used}, MaxCore: {maxcore_used} MB")
            print(f"Time Limit: {orca_slurm_time.value}")
            if orca_solvation_type.value != 'None':
                print(f"Solvation: {orca_solvation_type.value}({orca_solvent.value})")
            if saved_files:
                print(f"Extra files: {', '.join(saved_files)}")
            print()
            print("Check status in Job Status tab")
            reset_orca_builder()
        else:
            print("Error submitting job:")
            print(result.stderr or result.stdout)


def handle_orca_submit(button):
    """Handle SUBMIT ORCA JOB button click."""
    with orca_output:
        clear_output()

        job_name = orca_job_name.value.strip()
        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        preview_content = orca_preview.value.strip()
        if preview_content:
            inp_content = preview_content
        else:
            coords = strip_xyz_header(orca_coords.value)
            if not coords:
                print("Error: Coordinates or INP preview cannot be empty!")
                return
            inp_content = generate_orca_input()

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)

        inp_content = sanitize_orca_input(inp_content)

        inp_path = job_dir / f"{safe_job_name}.inp"
        inp_path.write_text(inp_content)

        saved_files = save_uploaded_files(job_dir)

        pal_used, maxcore_used = parse_inp_resources(inp_content)
        if pal_used is None:
            pal_used = orca_pal.value
        if maxcore_used is None:
            maxcore_used = orca_maxcore.value
        total_mem = pal_used * maxcore_used

        result = submit_job(
            job_dir=job_dir,
            mode='orca',
            job_name=safe_job_name,
            inp_file=f'{safe_job_name}.inp',
            time_limit=orca_slurm_time.value,
            ntasks=pal_used,
            mem_mb=total_mem
        )

        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("ORCA job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Job Name: {safe_job_name}")
            print(f"Directory: {job_dir}")
            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            print(f"PAL: {pal_used}, MaxCore: {maxcore_used} MB")
            print(f"Time Limit: {orca_slurm_time.value}")
            if orca_solvation_type.value != 'None':
                print(f"Solvation: {orca_solvation_type.value}({orca_solvent.value})")
            if saved_files:
                print(f"Extra files: {', '.join(saved_files)}")
            print()
            print("Check status in Job Status tab")
            reset_orca_builder()
        else:
            print("Error submitting job:")
            print(result.stderr or result.stdout)


# Connect buttons
orca_generate_btn.on_click(handle_orca_generate)
orca_save_btn.on_click(handle_orca_save)
orca_submit_btn.on_click(handle_orca_submit)

for w in [orca_method, orca_job_type, orca_basis, orca_dispersion, orca_ri,
          orca_aux_basis, orca_charge, orca_multiplicity, orca_pal, orca_maxcore,
          orca_coords, orca_additional, orca_solvation_type, orca_solvent,
          orca_print_mos, orca_print_basis, orca_autoaux]:
    w.observe(update_orca_preview, names='value')

orca_coords.observe(update_orca_molecule_view, names='value')
update_orca_molecule_view()
update_orca_preview()
_last_auto_keywords = _build_keyword_line()

# Layout
orca_row1 = widgets.HBox([orca_job_name])
orca_row2 = widgets.HBox([orca_coords])
orca_row3 = widgets.HBox([orca_charge, orca_multiplicity])
orca_row4 = widgets.HBox([orca_method, orca_job_type])
orca_row5 = widgets.HBox([orca_basis, orca_dispersion])
orca_row6 = widgets.HBox([orca_ri, orca_aux_basis])
orca_row6b = widgets.HBox([orca_autoaux])
orca_row_solv = widgets.HBox([orca_solvation_type, orca_solvent])
orca_row_opts = widgets.HBox([orca_print_mos, orca_print_basis])
orca_row7 = widgets.HBox([orca_additional])
orca_row8 = widgets.HBox([orca_pal, orca_maxcore])
orca_row9 = widgets.HBox([orca_slurm_time])
orca_row_upload = widgets.VBox(
    [orca_file_upload, orca_uploaded_files_label],
    layout=widgets.Layout(margin='0 0 -5px 120px')
)
orca_row_paths = widgets.HBox([orca_path_files])
orca_row_buttons = widgets.HBox(
    [orca_generate_btn, orca_save_btn, orca_submit_btn],
    layout=widgets.Layout(margin='0 0 0 120px')
)

orca_left = widgets.VBox([
    orca_row1,
    orca_row2,
    orca_row3,
    orca_row4,
    orca_row5,
    orca_row6,
    orca_row6b,
    orca_row_solv,
    orca_row_opts,
    orca_row7,
    orca_row8,
    orca_row9,
    orca_row_upload,
    orca_row_paths,
    orca_row_buttons,
    orca_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_right = widgets.VBox([
    widgets.Label("Live ORCA Input Preview (editable):"),
    orca_preview,
    widgets.HTML('<b>Molecule Preview:</b>', layout=widgets.Layout(margin='10px 0 0 0')),
    widgets.HBox([orca_mol_output], layout=widgets.Layout(justify_content='center', width='100%'))
], layout=widgets.Layout(width='50%', padding='10px'))

orca_builder_ui = widgets.HBox([orca_left, orca_right])

orca_builder_section = widgets.VBox([
    orca_builder_title,
    orca_builder_help,
    orca_builder_ui
], layout=widgets.Layout(width='100%', padding='10px', border='1px solid #4CAF50'))

tab4_content = orca_builder_section


# === Calculations Browser Tab ===
_calc_current_path = ''
_calc_file_content = ''
_calc_all_items = []  # All items in current directory
_calc_search_spans = []  # List of (start, end) positions
_calc_current_match = -1  # Current match index
_calc_selected_inp_path = None
_calc_selected_inp_base = ''
_calc_recalc_active = False
_calc_delete_current = False

calc_path_label = widgets.HTML(value='<b>📂 Path:</b> /', layout=widgets.Layout(width='100%', overflow_x='hidden'))

calc_back_btn = widgets.Button(description='⬆ Up', button_style='warning', layout=widgets.Layout(width='58px', height='26px'), disabled=True)
calc_home_btn = widgets.Button(description='🏠', button_style='info', layout=widgets.Layout(width='58px', height='26px'))
calc_refresh_btn = widgets.Button(description='🔄', layout=widgets.Layout(width='58px', height='26px'))
calc_delete_btn = widgets.Button(description='🗑 Delete', button_style='danger', layout=widgets.Layout(width='80px', height='26px'))
calc_delete_yes_btn = widgets.Button(description='Yes', button_style='danger', layout=widgets.Layout(width='60px', height='26px'))
calc_delete_no_btn = widgets.Button(description='No', layout=widgets.Layout(width='60px', height='26px'))
calc_delete_label = widgets.HTML('<b>Delete selected item?</b>')
calc_delete_confirm = widgets.HBox([
    calc_delete_label,
    calc_delete_yes_btn,
    calc_delete_no_btn
], layout=widgets.Layout(display='none', gap='6px', align_items='center'))
calc_delete_status = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))

# Folder search (filter files in directory)
calc_folder_search = widgets.Text(placeholder='Filter files...', continuous_update=True, layout=widgets.Layout(flex='1', height='28px', overflow_x='hidden', margin='0', padding='0'))

# Sort dropdown
calc_sort_dropdown = widgets.Dropdown(
    options=[('A-Z', 'name'), ('Newest', 'date_desc'), ('Oldest', 'date_asc')],
    value='name',
    layout=widgets.Layout(width='70px', height='28px', padding='0', margin='-27px 0 0 0')
)
calc_filter_sort_row = widgets.HBox([calc_folder_search, calc_sort_dropdown], layout=widgets.Layout(width='100%', margin='0 0 2px 0', gap='4px', align_items='center'))

# Sort dropdown
calc_sort_dropdown = widgets.Dropdown(
    options=[('A-Z', 'name'), ('Newest', 'date_desc'), ('Oldest', 'date_asc')],
    value='name',
    layout=widgets.Layout(width='30%', height='26px')
)
calc_filter_sort_row = widgets.HBox([calc_folder_search, calc_sort_dropdown], layout=widgets.Layout(width='100%', margin='0 0 8px 0', gap='4px'))

calc_file_list = widgets.Select(options=[], rows=22, layout=widgets.Layout(width='100%', height='520px', margin='-4px 0 0 0'))

# Content area - HTML with highlights and scroll
calc_content_area = widgets.HTML(value='', layout=widgets.Layout(width='100%', display='block', overflow_x='hidden'))

calc_mol_label = widgets.HTML(
    "<div style='height:26px; line-height:26px; margin:0 0 8px 0;'><b>🔬 Molecule Preview:</b></div>"
)
calc_mol_viewer = widgets.Output(layout=widgets.Layout(width='520px', height='520px', border='2px solid #1976d2', overflow='hidden', padding='0', border_radius='0'))

# === Multi-frame XYZ trajectory support ===
_calc_xyz_frames = []  # List of (comment, xyz_content) tuples for each frame
_calc_xyz_current_frame = [0]
_calc_xyz_viewer_id = [None]  # Store viewer ID for frame switching  # Use list to avoid global issues

calc_xyz_frame_label = widgets.HTML(value='', layout=widgets.Layout(margin='4px 0'))
calc_xyz_prev_btn = widgets.Button(
    description='◀ Prev',
    button_style='info',
    layout=widgets.Layout(width='80px', height='32px')
)
calc_xyz_next_btn = widgets.Button(
    description='Next ▶',
    button_style='info',
    layout=widgets.Layout(width='80px', height='32px')
)
calc_xyz_frame_input = widgets.BoundedIntText(
    value=1, min=1, max=1, step=1,
    layout=widgets.Layout(width='60px', height='28px')
)
calc_xyz_frame_total = widgets.HTML(value='<b>/ 1</b>', layout=widgets.Layout(width='40px'))
calc_xyz_copy_btn = widgets.Button(
    description='📋 Copy Coordinates',
    button_style='success',
    layout=widgets.Layout(width='160px', height='32px')
)
calc_xyz_controls = widgets.HBox([
    widgets.HTML('<b>Frame:</b>'),
    calc_xyz_frame_input,
    calc_xyz_frame_total,
    calc_xyz_copy_btn
], layout=widgets.Layout(display='none', gap='6px', margin='6px 0', align_items='center'))

# Coord file copy button (for single Turbomole coord files, no frames)
calc_coord_copy_btn = widgets.Button(
    description='📋 Copy Coordinates',
    button_style='success',
    layout=widgets.Layout(width='160px', height='32px')
)
calc_coord_controls = widgets.HBox([
    calc_coord_copy_btn
], layout=widgets.Layout(display='none', gap='6px', margin='6px 0', align_items='center'))

def parse_xyz_frames(content):
    """Parse a multi-frame XYZ file into a list of (comment, xyz_block) tuples."""
    frames = []
    lines = content.strip().split('\n')
    i = 0
    while i < len(lines):
        # First line should be atom count
        try:
            n_atoms = int(lines[i].strip())
        except (ValueError, IndexError):
            break
        if i + 1 >= len(lines):
            break
        comment = lines[i + 1].strip()
        # Extract coordinate lines
        coord_lines = []
        for j in range(n_atoms):
            if i + 2 + j < len(lines):
                coord_lines.append(lines[i + 2 + j])
        if len(coord_lines) == n_atoms:
            xyz_block = '\n'.join(coord_lines)
            frames.append((comment, xyz_block, n_atoms))
        i += 2 + n_atoms
    return frames


def calc_update_xyz_viewer(initial_load=False):
    """Update the viewer for the current frame."""
    if not _calc_xyz_frames:
        return
    idx = _calc_xyz_current_frame[0]
    if idx < 0 or idx >= len(_calc_xyz_frames):
        return
    comment, xyz_block, n_atoms = _calc_xyz_frames[idx]
    
    # Update frame display (suppress observer temporarily)
    calc_xyz_frame_input.unobserve(calc_on_xyz_input_change, names='value')
    calc_xyz_frame_input.value = idx + 1
    calc_xyz_frame_input.max = len(_calc_xyz_frames)
    calc_xyz_frame_input.observe(calc_on_xyz_input_change, names='value')
    calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
    # Update label with comment
    calc_xyz_frame_label.value = f"{html.escape(comment[:100])}{'...' if len(comment) > 100 else ''}"
    
    if initial_load:
        # Build complete trajectory XYZ (all frames concatenated)
        full_xyz = ""
        for comm, block, natoms in _calc_xyz_frames:
            full_xyz += f"{natoms}\n{comm}\n{block}\n"
        
        # Create viewer with unique ID
        calc_mol_viewer.clear_output()
        with calc_mol_viewer:
            viewer_id = "calc_trj_viewer"
            html_content = f"""
            <div id="{viewer_id}" style="width:520px;height:520px;position:relative;"></div>
            <script>
            (function() {{
                var viewer = $3Dmol.createViewer("{viewer_id}", {{backgroundColor: "white"}});
                var xyz = `{full_xyz}`;
                viewer.addModelsAsFrames(xyz, "xyz");
                viewer.setStyle({{}}, {{stick: {{radius: 0.1}}, sphere: {{scale: 0.25}}}});
                viewer.zoomTo();
                viewer.setFrame(0);
                viewer.render();
                window.calc_trj_viewer = viewer;
            }})();
            </script>
            """
            display(HTML(html_content))
    else:
        # Just switch frame via JavaScript - don't clear output!
        with calc_mol_viewer:
            display(HTML(f"""
            <script>
            if(window.calc_trj_viewer) {{
                window.calc_trj_viewer.setFrame({idx});
                window.calc_trj_viewer.render();
            }}
            </script>
            """))


def calc_on_xyz_input_change(change):
    """Jump to entered frame number."""
    new_val = change['new']
    if _calc_xyz_frames and 1 <= new_val <= len(_calc_xyz_frames):
        _calc_xyz_current_frame[0] = new_val - 1
        calc_update_xyz_viewer(initial_load=False)


def calc_on_xyz_copy(button):
    """Copy current frame coordinates to clipboard via JavaScript."""
    global _calc_xyz_current_frame
    if not _calc_xyz_frames:
        return
    idx = _calc_xyz_current_frame[0]
    if idx < 0 or idx >= len(_calc_xyz_frames):
        return
    comment, xyz_block, n_atoms = _calc_xyz_frames[idx]
    # Build full XYZ content for clipboard
    xyz_content = f"{n_atoms}\n{comment}\n{xyz_block}"
    # Escape for JavaScript
    escaped = xyz_content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = f"navigator.clipboard.writeText('{escaped}').then(() => console.log('Copied frame {idx+1}')).catch(err => console.error('Copy failed:', err));"
    _run_js(js_code)


calc_xyz_frame_input.observe(calc_on_xyz_input_change, names='value')
calc_xyz_copy_btn.on_click(calc_on_xyz_copy)

def calc_on_coord_copy(button):
    """Copy coord file content to clipboard via JavaScript."""
    global _calc_file_content
    if not _calc_file_content:
        return
    # Escape for JavaScript
    escaped = _calc_file_content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = f"navigator.clipboard.writeText('{escaped}').then(() => console.log('Copied coord')).catch(err => console.error('Copy failed:', err));"
    _run_js(js_code)

calc_coord_copy_btn.on_click(calc_on_coord_copy)

calc_copy_path_btn = widgets.Button(description='PATH', button_style='', layout=widgets.Layout(width='70px', height='26px'), disabled=True)

calc_copy_btn = widgets.Button(description='Copy', button_style='info', layout=widgets.Layout(width='80px', height='26px'), disabled=True)

# Report button - generates DOCX report via delfin --report docx
calc_report_btn = widgets.Button(description='Report', button_style='success', layout=widgets.Layout(width='80px', height='26px'), disabled=True)
calc_report_status = widgets.HTML(value='', layout=widgets.Layout(width='100%'))

def calc_update_report_btn():
    '''Enable Report button only if CONTROL.txt exists in current directory.'''
    global _calc_current_path
    current_dir = CALC_DIR / _calc_current_path if _calc_current_path else CALC_DIR
    control_file = current_dir / 'CONTROL.txt'
    calc_report_btn.disabled = not control_file.exists()

_calc_report_running = {}  # Track running reports by directory

def calc_on_report(button):
    '''Run delfin --report docx in current directory (background thread).'''
    global _calc_current_path, _calc_report_running
    import subprocess
    import threading
    
    current_dir = CALC_DIR / _calc_current_path if _calc_current_path else CALC_DIR
    dir_key = str(current_dir)
    
    if dir_key in _calc_report_running:
        calc_report_status.value = '<span style="color:orange;">Report already running for this folder...</span>'
        return
    
    _calc_report_running[dir_key] = True
    calc_report_status.value = f'<span style="color:blue;">Generating report in {current_dir.name}...</span>'
    
    def run_report():
        try:
            result = subprocess.run(
                ['delfin', '--report', 'docx'],
                cwd=str(current_dir),
                capture_output=True,
                text=True,
                timeout=600
            )
            if result.returncode == 0:
                calc_report_status.value = f'<span style="color:green;">Report generated in {current_dir.name}!</span>'
            else:
                calc_report_status.value = f'<span style="color:red;">Error in {current_dir.name}: {html.escape(result.stderr[:100])}</span>'
        except subprocess.TimeoutExpired:
            calc_report_status.value = f'<span style="color:red;">Timeout in {current_dir.name}</span>'
        except Exception as e:
            calc_report_status.value = f'<span style="color:red;">Error: {html.escape(str(e)[:100])}</span>'
        finally:
            _calc_report_running.pop(dir_key, None)
    
    thread = threading.Thread(target=run_report, daemon=True)
    thread.start()

calc_report_btn.on_click(calc_on_report)

def calc_on_content_copy(button):
    """Copy current file content to clipboard via JavaScript."""
    global _calc_file_content, _calc_recalc_active
    content = ''
    if _calc_recalc_active:
        content = calc_edit_area.value or ''
    if not content:
        content = _calc_file_content or ''
    if not content:
        return
    escaped = content.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code = f"navigator.clipboard.writeText('{escaped}').then(() => console.log('Copied file content')).catch(err => console.error('Copy failed:', err));"
    _run_js(js_code)

calc_copy_btn.on_click(calc_on_content_copy)

def calc_on_path_copy(button):
    """Copy the full path of the currently selected item to clipboard."""
    global _calc_current_path
    selected = calc_file_list.value
    if not selected:
        return
    name = selected[2:].strip()
    if _calc_current_path:
        full_path = str(CALC_DIR / _calc_current_path / name)
    else:
        full_path = str(CALC_DIR / name)
    # Show path as selectable input (works in VS Code + Jupyter)
    calc_path_display.value = f'<input type="text" value="{html.escape(full_path)}" onclick="this.select()" style="width:100%;font-family:monospace;font-size:12px;border:1px solid #aaa;padding:2px;background:#f8f8f8" readonly>'
    # Also try JS clipboard
    escaped = full_path.replace('\\', '\\\\').replace("'", "\\'")
    js_code = f"navigator.clipboard.writeText('{escaped}').then(() => console.log('Copied path')).catch(err => console.error('Copy failed:', err));"
    _run_js(js_code)

calc_copy_path_btn.on_click(calc_on_path_copy)

calc_mol_container = widgets.VBox([calc_mol_label, calc_mol_viewer], layout=widgets.Layout(display='none', width='100%', align_items='flex-start'))

calc_file_info = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))
calc_path_display = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))
calc_content_label = widgets.HTML(value='<b>📄 File Content:</b>', layout=widgets.Layout(width='100%', overflow_x='hidden', margin='8px 0 0 0'))
calc_view_toggle = widgets.ToggleButton(description='Visualize', value=False, disabled=True, button_style='warning', layout=widgets.Layout(width='110px', height='26px'))


calc_recalc_btn = widgets.Button(description='Recalc', button_style='warning', layout=widgets.Layout(width='80px', height='26px'), disabled=True)
calc_submit_recalc_btn = widgets.Button(description='Submit Recalc', button_style='success', layout=widgets.Layout(width='130px', height='26px'), disabled=True)
calc_recalc_time = widgets.Text(value='24:00:00', description='Time limit', layout=widgets.Layout(width='200px', height='26px'))
calc_recalc_status = widgets.HTML(value='', layout=widgets.Layout(width='100%', overflow_x='hidden'))
calc_edit_area = widgets.Textarea(value='', layout=widgets.Layout(width='100%', height='520px', display='none'))

# Content navigation
calc_top_btn = widgets.Button(description='⬆ Top', layout=widgets.Layout(width='95px', height='26px'))
calc_bottom_btn = widgets.Button(description='⬇ End', layout=widgets.Layout(width='95px', height='26px'))

# Content search with navigation
calc_search_options = [
    'ABSORPTION SPECTRUM',
    'ERROR',
    'FINAL SINGLE POINT ENERGY',
    'Final Gibbs free energy',
    'HURRAY',
    'JOB NUMBER ',
    'LOEWDIN POPULATION ANALYSIS',
    'LOEWDIN REDUCED ORBITAL CHARGES',
    'MULLIKEN POPULATION ANALYSIS',
    'MULLIKEN REDUCED ORBITAL CHARGES',
    'ORBITAL ENERGIES',
    'SCF CONVERGED AFTER',
    'TD-DFT EXCITED STATES',
    'Total Enthalpy',
    'WARNING'
]
calc_search_input = widgets.Text(
    placeholder='Search in file...',
    continuous_update=False,
    layout=widgets.Layout(width='140px', height='26px')
)
calc_search_suggest = widgets.Dropdown(
    options=['(Select)'] + calc_search_options,
    value='(Select)',
    layout=widgets.Layout(width='200px', height='26px')
)
calc_search_btn = widgets.Button(description='🔍', layout=widgets.Layout(width='85px', height='26px'))
calc_prev_btn = widgets.Button(description='◀', layout=widgets.Layout(width='58px', height='26px'), disabled=True)
calc_next_btn = widgets.Button(description='▶', layout=widgets.Layout(width='58px', height='26px'), disabled=True)
calc_search_result = widgets.HTML(value='', layout=widgets.Layout(width='180px', min_width='180px'))

# === OCCUPIER Override Widgets ===
calc_options_dropdown = widgets.Dropdown(
    options=['(Options)'],
    value='(Options)',
    layout=widgets.Layout(width='150px', height='26px', visibility='hidden')
)
calc_override_input = widgets.Text(
    value='',
    placeholder='STAGE=INDEX',
    layout=widgets.Layout(width='140px', height='26px', visibility='hidden')
)
calc_override_time = widgets.Text(
    value='08:00:00',
    placeholder='HH:MM:SS',
    layout=widgets.Layout(width='80px', height='26px', visibility='hidden')
)
calc_override_btn = widgets.Button(
    description='Submit',
    button_style='success',
    layout=widgets.Layout(width='70px', height='26px', visibility='hidden')
)
calc_override_status = widgets.HTML(value='', layout=widgets.Layout(width='100%', visibility='hidden'))

# === OCCUPIER Override Handler ===
def calc_update_options_dropdown():
    """Update options dropdown based on selected file."""
    selected = calc_file_list.value
    sel_lower = selected.lower() if selected else ''
    if selected and 'OCCUPIER.txt' in selected:
        calc_options_dropdown.options = ['(Options)', 'Override']
        calc_options_dropdown.value = '(Options)'
        calc_options_dropdown.layout.visibility = 'visible'
    elif selected and ('CONTROL.txt' in selected or sel_lower.endswith('.inp')):
        calc_options_dropdown.options = ['(Options)', 'Recalc']
        calc_options_dropdown.value = '(Options)'
        calc_options_dropdown.layout.visibility = 'visible'
    else:
        calc_options_dropdown.options = ['(Options)']
        calc_options_dropdown.value = '(Options)'
        calc_options_dropdown.layout.visibility = 'hidden'
        calc_override_input.layout.visibility = 'hidden'
        calc_override_time.layout.visibility = 'hidden'
        calc_override_btn.layout.visibility = 'hidden'
        calc_override_status.layout.visibility = 'hidden'
        calc_override_status.value = ''
        calc_edit_area.layout.display = 'none'
        calc_content_area.layout.display = 'block'

def calc_on_options_change(change):
    """Show/hide override input when Override or Recalc is selected."""
    global _calc_file_content
    if change['new'] == 'Override':
        calc_override_input.layout.visibility = 'visible'
        calc_override_time.layout.visibility = 'visible'
        calc_override_btn.layout.visibility = 'visible'
        calc_override_status.layout.visibility = 'visible'
        calc_override_status.value = ''
        calc_edit_area.layout.display = 'none'
        calc_content_area.layout.display = 'block'
        # Pre-fill with stage name from filename
        selected = calc_file_list.value
        if selected:
            name = selected[2:].strip()
            # Extract stage name: e.g. "red_step_2_OCCUPIER.txt" -> "red_step_2"
            stage_name = name.replace('_OCCUPIER.txt', '').replace('OCCUPIER.txt', '')
            if stage_name:
                calc_override_input.value = f'{stage_name}='
            else:
                calc_override_input.value = ''
    elif change['new'] == 'Recalc':
        calc_override_input.layout.visibility = 'hidden'
        calc_override_time.layout.visibility = 'visible'
        calc_override_btn.layout.visibility = 'visible'
        calc_override_status.layout.visibility = 'visible'
        calc_override_status.value = ''
        # Show editable content
        calc_edit_area.value = _calc_file_content
        calc_edit_area.layout.display = 'block'
        calc_content_area.layout.display = 'none'
    else:
        calc_override_input.layout.visibility = 'hidden'
        calc_override_time.layout.visibility = 'hidden'
        calc_override_btn.layout.visibility = 'hidden'
        calc_override_status.layout.visibility = 'hidden'
        calc_override_status.value = ''
        calc_edit_area.layout.display = 'none'
        calc_content_area.layout.display = 'block'

def calc_find_workspace_root(current_path):
    """Find workspace root (directory containing CONTROL.txt) from current path."""
    if not current_path:
        return None
    
    # Start from current directory and search upwards
    path_parts = current_path.split('/')
    for i in range(len(path_parts), 0, -1):
        test_path = '/'.join(path_parts[:i])
        control_file = CALC_DIR / test_path / 'CONTROL.txt'
        if control_file.exists():
            return test_path
    return None

def calc_on_override_start(button):
    """Execute delfin --occupier-override, DELFIN recalc, or ORCA recalc."""
    global _calc_current_path, _calc_file_content
    
    is_recalc = calc_options_dropdown.value == 'Recalc'
    selected = calc_file_list.value
    selected_name = selected[2:].strip() if selected and len(selected) > 2 else ''
    is_inp_recalc = bool(is_recalc and selected_name.lower().endswith('.inp'))
    
    if not is_recalc:
        override_value = calc_override_input.value.strip()
        if not override_value or '=' not in override_value:
            calc_override_status.value = '<span style="color:#d32f2f;">Enter STAGE=INDEX (e.g., red_step_2=1)</span>'
            return

    # ORCA recalc for .inp selection
    if is_inp_recalc:
        inp_path = None
        if _calc_selected_inp_path and Path(_calc_selected_inp_path).exists():
            inp_path = Path(_calc_selected_inp_path)
        elif selected_name:
            inp_path = (CALC_DIR / _calc_current_path / selected_name) if _calc_current_path else (CALC_DIR / selected_name)
            if not inp_path.exists():
                inp_path = None
        if inp_path is None:
            calc_override_status.value = '<span style="color:#d32f2f;">No .inp file selected.</span>'
            return

        inp_content = calc_edit_area.value or inp_path.read_text()
        if not inp_content.strip():
            calc_override_status.value = '<span style="color:#d32f2f;">Input content is empty.</span>'
            return

        job_dir = inp_path.parent
        base_stem = inp_path.stem
        base_name = calc_get_recalc_base_name(base_stem)
        next_idx = calc_next_recalc_index(job_dir, base_name)
        job_name = f"{base_name}_recalc_{next_idx}"

        try:
            new_inp = job_dir / f"{job_name}.inp"
            new_inp.write_text(inp_content)

            slurm_time = calc_override_time.value.strip() or '24:00:00'

            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            total_mem = pal_used * maxcore_used

            result = submit_job(
                job_dir=job_dir,
                mode='orca',
                job_name=job_name,
                inp_file=f'{job_name}.inp',
                time_limit=slurm_time,
                ntasks=pal_used,
                mem_mb=total_mem
            )
        except Exception as exc:
            calc_override_status.value = f'<span style="color:#d32f2f;">Error: {html.escape(str(exc))}</span>'
            return

        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
            calc_override_status.value = (
                f'<span style="color:#2e7d32;">Submitted {html.escape(job_name)} (ID: {html.escape(job_id)})</span>'
            )
        else:
            msg = result.stderr or result.stdout or 'Unknown error'
            calc_override_status.value = f'<span style="color:#d32f2f;">{html.escape(msg)}</span>'
        return
    
    # Find workspace root (where CONTROL.txt is located)
    if not _calc_current_path:
        calc_override_status.value = '<span style="color:#d32f2f;">No folder selected</span>'
        return
    
    workspace_root = calc_find_workspace_root(_calc_current_path)
    if not workspace_root:
        calc_override_status.value = '<span style="color:#d32f2f;">No CONTROL.txt found in parent directories</span>'
        return
    
    workspace_path = CALC_DIR / workspace_root
    
    # For Recalc: save edited CONTROL.txt first
    if is_recalc:
        try:
            control_path = workspace_path / 'CONTROL.txt'
            control_path.write_text(calc_edit_area.value)
            _calc_file_content = calc_edit_area.value
            calc_render_content(scroll_to='top')
        except Exception as e:
            calc_override_status.value = f'<span style="color:#d32f2f;">Error saving CONTROL.txt: {e}</span>'
            return
    
    calc_override_status.value = '<span style="color:#1976d2;">Submitting job...</span>'
    
    try:
        # Derive PAL/maxcore from CONTROL.txt first, fallback to .inp
        pal_used = None
        maxcore_used = None
        control_path = workspace_path / 'CONTROL.txt'
        if control_path.exists():
            pal_used, maxcore_used = parse_resource_settings(control_path.read_text())

        if pal_used is None or maxcore_used is None:
            inp_candidate = None
            try:
                if _calc_selected_inp_path and Path(_calc_selected_inp_path).exists():
                    inp_candidate = Path(_calc_selected_inp_path)
            except Exception:
                inp_candidate = None

            if inp_candidate is None:
                inp_files = sorted(workspace_path.glob('*.inp'))
                inp_candidate = inp_files[0] if inp_files else None

            if inp_candidate is not None and inp_candidate.exists():
                pal_inp, maxcore_inp = parse_inp_resources(inp_candidate.read_text())
                if pal_used is None:
                    pal_used = pal_inp
                if maxcore_used is None:
                    maxcore_used = maxcore_inp

        if pal_used is None or maxcore_used is None:
            calc_override_status.value = '<span style="color:#d32f2f;">PAL/maxcore not found in CONTROL.txt or .inp</span>'
            return

        mem_mb = pal_used * maxcore_used

        if is_recalc:
            # Submit recalc job
            job_name = f'recalc_{workspace_root.replace("/", "_")}'
            env_vars = f"DELFIN_MODE=delfin-recalc,DELFIN_JOB_NAME={job_name}"
        else:
            # Submit occupier-override job
            job_name = f'occ_{override_value.replace("=", "_")}'
            env_vars = f"DELFIN_MODE=delfin-recalc-override,DELFIN_JOB_NAME={job_name},DELFIN_OVERRIDE={override_value}"
        
        if is_recalc:
            result = local_submit(
                job_dir=workspace_path,
                mode='delfin-recalc',
                job_name=job_name,
                time_limit=calc_override_time.value,
                pal=pal_used,
                maxcore=maxcore_used
            )
        else:
            result = local_submit(
                job_dir=workspace_path,
                mode='delfin-recalc-override',
                job_name=job_name,
                time_limit=calc_override_time.value,
                override=override_value,
                pal=pal_used,
                maxcore=maxcore_used
            )
        
        if result.returncode == 0:
            calc_override_status.value = f'<span style="color:#388e3c;">Submitted from {workspace_root}: {result.stdout.strip()}</span>'
        else:
            calc_override_status.value = f'<span style="color:#d32f2f;">Error: {result.stderr.strip()}</span>'
    except Exception as e:
        calc_override_status.value = f'<span style="color:#d32f2f;">Error: {e}</span>'

calc_options_dropdown.observe(calc_on_options_change, names='value')
calc_override_btn.on_click(calc_on_override_start)

def calc_update_path_display():
    global _calc_current_path
    display_path = '/' + _calc_current_path if _calc_current_path else '/'
    calc_path_label.value = f'<b>📂 Path:</b> <code style="background:#eee;padding:2px 5px; white-space:nowrap; overflow:hidden; text-overflow:ellipsis; display:block; max-width:100%;" title="{display_path}">{display_path}</code>'
    calc_back_btn.disabled = (_calc_current_path == '')

def calc_set_message(message):
    calc_content_area.value = (
        "<div style='height:520px; overflow-y:auto; border:1px solid #ddd; padding:6px;"
        " font-family:monospace; white-space:pre; background:#fafafa;'>"
        f"{html.escape(message)}"
        "</div>"
    )

def calc_clear_selection():
    calc_file_list.value = None
    calc_file_list.index = None


def calc_delete_hide_confirm():
    global _calc_delete_current
    calc_delete_confirm.layout.display = 'none'
    calc_delete_status.value = ''
    _calc_delete_current = False


def calc_on_delete_click(button):
    global _calc_delete_current
    if not calc_file_list.value or calc_file_list.value.startswith('('):
        # No selection: delete current folder
        if _calc_current_path:
            calc_delete_label.value = f'<b>Delete current folder?</b> <code>{html.escape(_calc_current_path)}</code>'
            _calc_delete_current = True
            calc_delete_confirm.layout.display = 'flex'
        else:
            calc_delete_status.value = '<span style="color:#d32f2f;">No item selected.</span>'
        return
    calc_delete_label.value = '<b>Delete selected item?</b>'
    _calc_delete_current = False
    calc_delete_confirm.layout.display = 'flex'


def calc_on_delete_yes(button):
    global _calc_current_path, _calc_delete_current
    delete_current = _calc_delete_current
    if delete_current:
        if not _calc_current_path:
            calc_delete_status.value = '<span style="color:#d32f2f;">No current folder.</span>'
            calc_delete_confirm.layout.display = 'none'
            _calc_delete_current = False
            return
        full_path = CALC_DIR / _calc_current_path
        name = _calc_current_path
    else:
        selected = calc_file_list.value
        if not selected or selected.startswith('('):
            calc_delete_status.value = '<span style="color:#d32f2f;">No item selected.</span>'
            calc_delete_confirm.layout.display = 'none'
            _calc_delete_current = False
            return
        name = selected[2:].strip()
        full_path = (CALC_DIR / _calc_current_path / name) if _calc_current_path else (CALC_DIR / name)
    try:
        if full_path.is_dir():
            shutil.rmtree(full_path)
        else:
            full_path.unlink()
        calc_delete_status.value = f'<span style="color:#2e7d32;">Deleted: {html.escape(name)}</span>'
    except Exception as exc:
        calc_delete_status.value = f'<span style="color:#d32f2f;">Delete failed: {html.escape(str(exc))}</span>'
    calc_delete_confirm.layout.display = 'none'
    _calc_delete_current = False
    if delete_current:
        _calc_current_path = _calc_current_path.rsplit('/', 1)[0] if '/' in _calc_current_path else ''
    # Preserve filter and refresh
    saved_filter = calc_folder_search.value
    calc_list_directory()
    if saved_filter:
        calc_folder_search.value = saved_filter
        calc_filter_file_list()
    calc_clear_selection()


def calc_on_delete_no(button):
    calc_delete_hide_confirm()


def calc_update_view():
    show_mol = calc_view_toggle.value
    if show_mol:
        calc_mol_container.layout.display = 'block'
        calc_content_area.layout.display = 'none'
        calc_edit_area.layout.display = 'none'
        calc_content_label.layout.display = 'none'
        if 'calc_content_toolbar' in globals():
            calc_content_toolbar.layout.display = 'none'
        if 'calc_recalc_toolbar' in globals():
            calc_recalc_toolbar.layout.display = 'none'
    else:
        calc_mol_container.layout.display = 'none'
        calc_content_label.layout.display = 'block'
        if 'calc_content_toolbar' in globals():
            calc_content_toolbar.layout.display = 'none' if _calc_recalc_active else 'flex'
        if _calc_recalc_active:
            calc_content_area.layout.display = 'none'
            calc_edit_area.layout.display = 'block'
            if 'calc_recalc_toolbar' in globals():
                calc_recalc_toolbar.layout.display = 'flex'
        else:
            calc_content_area.layout.display = 'block'
            calc_edit_area.layout.display = 'none'
            if 'calc_recalc_toolbar' in globals():
                calc_recalc_toolbar.layout.display = 'none'




def calc_reset_recalc_state():
    global _calc_selected_inp_path, _calc_selected_inp_base, _calc_recalc_active
    _calc_selected_inp_path = None
    _calc_selected_inp_base = ''
    _calc_recalc_active = False
    calc_recalc_btn.disabled = True
    calc_submit_recalc_btn.disabled = True
    calc_recalc_status.value = ''
    calc_edit_area.value = ''
    calc_edit_area.layout.display = 'none'
    if 'calc_recalc_toolbar' in globals():
        calc_recalc_toolbar.layout.display = 'none'
    calc_content_label.value = '<b>📄 File Content:</b>'


def calc_get_recalc_base_name(stem):
    match = re.match(r'^(.*)_recalc_(\d+)$', stem)
    return match.group(1) if match else stem


def calc_next_recalc_index(job_dir, base_name):
    pattern = re.compile(rf'^{re.escape(base_name)}_recalc_(\d+)\.(?:inp|out)$')
    max_idx = 0
    for entry in job_dir.iterdir():
        match = pattern.match(entry.name)
        if match:
            max_idx = max(max_idx, int(match.group(1)))
    return max_idx + 1


def calc_on_recalc_click(button):
    global _calc_recalc_active
    if not _calc_selected_inp_path or not _calc_selected_inp_path.exists():
        calc_recalc_status.value = '<span style="color:#d32f2f;">No .inp file selected.</span>'
        return
    try:
        content = _calc_selected_inp_path.read_text()
    except Exception as exc:
        calc_recalc_status.value = f'<span style="color:#d32f2f;">Error reading file: {html.escape(str(exc))}</span>'
        return
    calc_edit_area.value = content
    _calc_recalc_active = True
    calc_submit_recalc_btn.disabled = False
    calc_view_toggle.value = False
    calc_content_label.value = '<b>📄 File Content (editable):</b>'
    calc_update_view()


def calc_on_submit_recalc(button):
    if not _calc_selected_inp_path or not _calc_selected_inp_path.exists():
        calc_recalc_status.value = '<span style="color:#d32f2f;">No .inp file selected.</span>'
        return
    inp_content = calc_edit_area.value
    if not inp_content.strip():
        calc_recalc_status.value = '<span style="color:#d32f2f;">Input content is empty.</span>'
        return

    job_dir = _calc_selected_inp_path.parent
    base_stem = _calc_selected_inp_path.stem
    base_name = calc_get_recalc_base_name(base_stem)
    next_idx = calc_next_recalc_index(job_dir, base_name)
    job_name = f"{base_name}_recalc_{next_idx}"

    try:
        inp_path = job_dir / f"{job_name}.inp"
        inp_path.write_text(inp_content)

        slurm_time = calc_recalc_time.value.strip() or '24:00:00'
        
        # Parse resources from input
        pal_used, maxcore_used = parse_inp_resources(inp_content)
        if pal_used is None:
            pal_used = orca_pal.value
        if maxcore_used is None:
            maxcore_used = orca_maxcore.value
        total_mem = pal_used * maxcore_used

        result = submit_job(
            job_dir=job_dir,
            mode='orca',
            job_name=job_name,
            inp_file=f'{job_name}.inp',
            time_limit=slurm_time,
            ntasks=pal_used,
            mem_mb=total_mem
        )
    except Exception as exc:
        calc_recalc_status.value = f'<span style="color:#d32f2f;">Error: {html.escape(str(exc))}</span>'
        return

    if result.returncode == 0:
        job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
        calc_recalc_status.value = (
            f'<span style="color:#2e7d32;">Submitted {html.escape(job_name)} (ID: {html.escape(job_id)})</span>'
        )
    else:
        msg = result.stderr or result.stdout or 'Unknown error'
        calc_recalc_status.value = f'<span style="color:#d32f2f;">{html.escape(msg)}</span>'

def calc_run_js(script):
    if not script:
        return
    _run_js(script)


def calc_scroll_to(target):
    if target == 'top':
        js = """
        setTimeout(function(){
            const box = document.getElementById('calc-content-box');
            if (box) { box.scrollTop = 0; }
        }, 0);
        """
        calc_run_js(js)
        return
    if target == 'bottom':
        js = """
        setTimeout(function(){
            const box = document.getElementById('calc-content-box');
            if (box) { box.scrollTop = box.scrollHeight; }
        }, 0);
        """
        calc_run_js(js)
        return
    if target == 'match' and _calc_current_match >= 0:
        js = """
        setTimeout(function(){
            const box = document.getElementById('calc-content-box');
            const el = document.getElementById('calc-current-match');
            if (!box || !el) return;
            const boxRect = box.getBoundingClientRect();
            const elRect = el.getBoundingClientRect();
            const delta = (elRect.top - boxRect.top) - (box.clientHeight / 2);
            box.scrollTop += delta;
        }, 0);
        """
        calc_run_js(js)




def calc_apply_highlight(query, current_index):
    if query is None:
        query = ''
    js = r"""
    (function() {
        const box = document.getElementById('calc-content-box');
        const el = document.getElementById('calc-content-text');
        if (!box || !el) return;
        const text = el.textContent || '';
        const q = __QUERY__;
        if (!q) {
            el.textContent = text;
            return;
        }
        function esc(s) {
            return s.replace(/[&<>"]/g, function(c) {
                return {'&':'&amp;','<':'&lt;','>':'&gt;','"':'&quot;'}[c];
            });
        }
        function escapeRegExp(s) {
            return s.replace(/[.*+?^${}()|[\\]\\]/g, '\\$&');
        }
        const re = new RegExp(escapeRegExp(q), 'gi');
        let html = '';
        let last = 0;
        let i = 0;
        let m;
        while ((m = re.exec(text)) !== null) {
            html += esc(text.slice(last, m.index));
            const cls = (i === __INDEX__) ? 'calc-match current' : 'calc-match';
            const id = (i === __INDEX__) ? 'calc-current-match' : '';
            html += `<mark class="${cls}" ${id ? 'id=\"' + id + '\"' : ''}>${esc(m[0])}</mark>`;
            last = m.index + m[0].length;
            i++;
        }
        html += esc(text.slice(last));
        el.innerHTML = html;
    })();
    """
    js = js.replace('__QUERY__', repr(query)).replace('__INDEX__', str(current_index))
    calc_run_js(js)


def calc_extract_orca_xyz_block(text):
    lines = text.splitlines()
    start_idx = None
    for i, line in enumerate(lines):
        if re.match(r'^\s*\*\s*xyz\b', line, flags=re.IGNORECASE):
            start_idx = i + 1
            break
    if start_idx is None:
        return None
    coords = []
    for line in lines[start_idx:]:
        if re.match(r'^\s*\*', line):
            break
        if not line.strip():
            continue
        coords.append(line.rstrip())
    return coords if coords else None


def calc_build_xyz_from_input(text, title='input.txt'):
    coords = calc_extract_orca_xyz_block(text)
    if coords:
        atom_count = len(coords)
        return f"{atom_count}\n{title}\n" + '\n'.join(coords)
    lines = [ln for ln in text.strip().splitlines() if ln.strip()]
    if not lines:
        return None
    # If first line is an integer atom count, assume header exists
    if re.fullmatch(r'\d+', lines[0].strip()):
        return '\n'.join(lines)
    atom_count = len(lines)
    return f"{atom_count}\n{title}\n" + '\n'.join(lines)



def calc_render_content(scroll_to=None):
    global _calc_file_content, _calc_search_spans, _calc_current_match
    if not _calc_file_content:
        calc_set_message('Select a file...')
        return

    text = _calc_file_content
    calc_content_area.value = (
        "<style>"
        ".calc-match { background: #fff59d; padding: 0 2px; }"
        ".calc-match.current { background: #ffcc80; }"
        "</style>"
        "<div id='calc-content-box' style='height:520px; overflow-y:auto; overflow-x:hidden; border:1px solid #ddd; padding:6px; background:#fafafa; width:100%; box-sizing:border-box;'>"
        "<div id='calc-content-text' style='white-space:pre-wrap; overflow-wrap:anywhere; word-break:break-word; font-family:monospace; font-size:12px; line-height:1.3;'>"
        f"{html.escape(text)}"
        "</div></div>"
    )
    if scroll_to:
        calc_scroll_to(scroll_to)
def calc_filter_file_list(change=None):
    """Filter file list based on search input."""
    global _calc_all_items
    query = calc_folder_search.value.strip().lower()
    if not query:
        calc_file_list.options = _calc_all_items
    else:
        filtered = [item for item in _calc_all_items if query in item.lower()]
        calc_file_list.options = filtered if filtered else ['(No matches)']

def calc_orca_terminated_normally(path):
    """Check for ORCA termination marker in the tail of a text file."""
    try:
        with path.open('rb') as f:
            f.seek(0, 2)
            size = f.tell()
            tail_size = 20000
            f.seek(max(0, size - tail_size))
            tail = f.read()
        text = tail.decode('utf-8', errors='ignore')
        return '****ORCA TERMINATED NORMALLY****' in text
    except Exception:
        return False

def calc_list_directory():
    global _calc_current_path, _calc_file_content, _calc_all_items, _calc_search_spans, _calc_current_match
    _calc_file_content = ''
    _calc_all_items = []
    _calc_search_spans = []
    _calc_current_match = -1
    calc_file_list.options = []
    calc_mol_viewer.clear_output()
    calc_mol_container.layout.display = 'none'
    calc_file_info.value = ''
    calc_path_display.value = ''
    calc_search_result.value = ''
    calc_folder_search.value = ''
    calc_prev_btn.disabled = True
    calc_next_btn.disabled = True
    calc_view_toggle.value = False
    calc_view_toggle.disabled = True
    calc_copy_btn.disabled = True
    calc_copy_path_btn.disabled = True
    calc_update_view()

    calc_set_message('Select a file...')

    current_dir = CALC_DIR / _calc_current_path if _calc_current_path else CALC_DIR

    if not current_dir.exists():
        calc_file_list.options = ['(Folder not found)']
        return

    items = []
    try:
        sort_mode = calc_sort_dropdown.value
        if sort_mode == 'date_desc':
            entries = sorted(current_dir.iterdir(), key=lambda x: (not x.is_dir(), -x.stat().st_mtime if x.exists() else 0))
        elif sort_mode == 'date_asc':
            entries = sorted(current_dir.iterdir(), key=lambda x: (not x.is_dir(), x.stat().st_mtime if x.exists() else 0))
        else:  # name
            entries = sorted(current_dir.iterdir(), key=lambda x: (not x.is_dir(), x.name.lower()))
        for entry in entries:
            if entry.is_dir():
                items.append(f'📁 {entry.name}')
            else:
                suffix = entry.suffix.lower()
                if suffix == '.xyz':
                    items.append(f'🔬 {entry.name}')
                elif suffix == '.png':
                    items.append(f'🖼 {entry.name}')
                elif suffix in ['.out', '.log']:
                    is_delfin = entry.name.lower().startswith('delfin_')
                    ok = True if is_delfin else calc_orca_terminated_normally(entry)
                    icon = '📄' if ok else '❌'
                    items.append(f'{icon} {entry.name}')
                elif suffix in ['.inp', '.sh']:
                    items.append(f'📝 {entry.name}')
                elif suffix in ['.cube', '.cub']:
                    items.append(f'🧊 {entry.name}')
                elif suffix in ['.gbw', '.cis', '.densities']:
                    items.append(f'💾 {entry.name}')
                elif suffix in ['.doc', '.docx']:
                    items.append(f'📃 {entry.name}')
                else:
                    items.append(f'📄 {entry.name}')
    except PermissionError:
        items = ['(Permission denied)']

    _calc_all_items = items if items else ['(Empty folder)']
    calc_file_list.options = _calc_all_items
    calc_update_path_display()
    calc_update_report_btn()

def calc_on_back(b):
    global _calc_current_path
    if _calc_current_path:
        parts = _calc_current_path.split('/')
        _calc_current_path = '/'.join(parts[:-1])
        calc_list_directory()

def calc_on_home(b):
    global _calc_current_path
    _calc_current_path = ''
    calc_list_directory()

def calc_on_refresh(b):
    calc_list_directory()

def calc_go_top(b):
    """Scroll to beginning of file."""
    if _calc_file_content:
        calc_render_content(scroll_to='top')

def calc_go_bottom(b):
    """Scroll to end of file."""
    if _calc_file_content:
        calc_render_content(scroll_to='bottom')

def calc_pos_to_line_col(pos):
    if pos < 0:
        return 0, 0
    line = _calc_file_content.count('\n', 0, pos) + 1
    last_nl = _calc_file_content.rfind('\n', 0, pos)
    col = pos + 1 if last_nl == -1 else pos - last_nl
    return line, col

def calc_update_nav_buttons():
    if not _calc_search_spans:
        calc_prev_btn.disabled = True
        calc_next_btn.disabled = True
        return
    calc_prev_btn.disabled = (_calc_current_match <= 0)
    calc_next_btn.disabled = (_calc_current_match >= len(_calc_search_spans) - 1)

def calc_update_search_result():
    if not _calc_search_spans:
        calc_search_result.value = ''
        return
    if _calc_current_match < 0:
        calc_search_result.value = f'<span style="color:green;">{len(_calc_search_spans)} matches</span>'
        return
    start, _ = _calc_search_spans[_calc_current_match]
    line, col = calc_pos_to_line_col(start)
    calc_search_result.value = (
        f'<b>{_calc_current_match + 1}/{len(_calc_search_spans)}</b> '
        f'<span style="color:#555;">(line {line}, col {col})</span>'
    )

def calc_do_search(b=None):
    """Search in file content."""
    global _calc_file_content, _calc_search_spans, _calc_current_match
    query = calc_search_input.value.strip()
    # If input empty but dropdown has value, use dropdown
    if not query and calc_search_suggest.value and calc_search_suggest.value != '(Select)':
        query = calc_search_suggest.value
        calc_search_input.value = query
    _calc_search_spans = []
    _calc_current_match = -1

    if not query or not _calc_file_content:
        calc_search_result.value = ''
        calc_update_nav_buttons()
        calc_apply_highlight('', -1)
        return

    pattern = re.compile(re.escape(query), re.IGNORECASE)
    _calc_search_spans = [match.span() for match in pattern.finditer(_calc_file_content)]

    if not _calc_search_spans:
        calc_search_result.value = '<span style="color:red;">0 matches</span>'
        calc_update_nav_buttons()
        calc_apply_highlight('', -1)
        return

    _calc_current_match = 0
    calc_update_nav_buttons()
    calc_update_search_result()
    calc_apply_highlight(query, _calc_current_match)
    calc_scroll_to('match')



def calc_on_suggest(change):
    value = change['new']
    if not value or value == '(Select)':
        return
    calc_search_input.value = value

def calc_show_match():
    """Display current match and scroll to it."""
    if not _calc_search_spans or _calc_current_match < 0:
        return
    calc_update_nav_buttons()
    calc_update_search_result()
    calc_apply_highlight(calc_search_input.value.strip(), _calc_current_match)
    calc_scroll_to('match')

def calc_prev_match(b):
    global _calc_current_match, _calc_search_spans
    if _calc_search_spans and _calc_current_match > 0:
        _calc_current_match -= 1
        calc_show_match()

def calc_next_match(b):
    global _calc_current_match, _calc_search_spans
    if _calc_search_spans and _calc_current_match < len(_calc_search_spans) - 1:
        _calc_current_match += 1
        calc_show_match()

def calc_on_select(change):
    global _calc_current_path, _calc_file_content, _calc_search_spans, _calc_current_match, _calc_selected_inp_path, _calc_selected_inp_base, _calc_xyz_frames, _calc_xyz_current_frame
    selected = change['new']
    if not selected or selected.startswith('('):
        return

    name = selected[2:].strip()
    full_path = (CALC_DIR / _calc_current_path / name) if _calc_current_path else (CALC_DIR / name)

    # Navigate into directory
    if full_path.is_dir():
        _calc_current_path = f'{_calc_current_path}/{name}' if _calc_current_path else name
        calc_list_directory()
        return

    # Reset search state
    _calc_search_spans = []
    _calc_current_match = -1
    calc_search_result.value = ''
    calc_search_input.value = ''
    calc_update_nav_buttons()
    calc_delete_hide_confirm()
    calc_update_options_dropdown()

    # Show file
    calc_mol_viewer.clear_output()
    calc_mol_container.layout.display = 'none'
    calc_content_area.layout.display = 'block'
    calc_view_toggle.value = False
    calc_view_toggle.disabled = True
    calc_file_info.value = ''
    calc_path_display.value = ''
    _calc_file_content = ''
    calc_reset_recalc_state()
    # Reset xyz trajectory controls (using module-level globals)
    _calc_xyz_frames.clear()
    _calc_xyz_current_frame[0] = 0
    calc_xyz_controls.layout.display = 'none'
    calc_coord_controls.layout.display = 'none'
    calc_xyz_frame_label.value = ''
    calc_xyz_frame_input.value = 1
    calc_xyz_frame_input.max = 1
    calc_xyz_frame_total.value = '<b>/ 1</b>'
    

    if not full_path.exists():
        calc_set_message('File not found')
        return

    # File info
    size = full_path.stat().st_size
    size_str = f'{size/(1024*1024):.2f} MB' if size > 1024*1024 else (f'{size/1024:.2f} KB' if size > 1024 else f'{size} B')

    suffix = full_path.suffix.lower()
    name_lower = full_path.name.lower()

    # Turbomole coord: show molecule viewer
    if name_lower == 'coord':
        calc_view_toggle.disabled = False
        calc_view_toggle.value = False
        calc_update_view()
        try:
            content = full_path.read_text()
            _calc_file_content = content
            calc_render_content(scroll_to='top')
            calc_copy_btn.disabled = False
            calc_copy_path_btn.disabled = False
            xyz_data = coord_to_xyz(content)
            if xyz_data:
                calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, Turbomole coord)'
                _calc_xyz_frames.clear()
                _calc_xyz_frames.append(xyz_data)
                _calc_xyz_current_frame[0] = 0
                calc_xyz_controls.layout.display = 'none'
                calc_coord_controls.layout.display = 'flex'
                with calc_mol_viewer:
                    clear_output(wait=True)
                    view = py3Dmol.view(width=520, height=520)
                    view.addModel(xyz_data, 'xyz')
                    view.setStyle({}, {'stick': {'radius': 0.15}, 'sphere': {'scale': 0.25}})
                    view.zoomTo()
                    view.show()
            else:
                calc_file_info.value = f'<b>{html.escape(name)}</b> (could not parse)'
        except Exception as e:
            calc_file_info.value = f'<b>{html.escape(name)}</b> (error: {e})'
        return

    # XYZ: show molecule viewer (with trajectory support)
    if suffix == '.xyz':
        calc_view_toggle.disabled = False
        calc_view_toggle.value = False
        calc_update_view()
        try:
            content = full_path.read_text()
            _calc_file_content = content
            calc_render_content(scroll_to='top')
            calc_copy_btn.disabled = False
            calc_copy_path_btn.disabled = False
            lines = content.split('\n')
            
            # Parse multi-frame XYZ
            _calc_xyz_frames.clear()
            _calc_xyz_frames.extend(parse_xyz_frames(content))
            n_frames = len(_calc_xyz_frames)
            
            if n_frames > 1:
                # Multi-frame trajectory
                calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, {n_frames} frames)'
                _calc_xyz_current_frame[0] = 0
                calc_xyz_frame_input.max = n_frames
                calc_xyz_frame_input.value = 1
                calc_xyz_frame_total.value = f"<b>/ {n_frames}</b>"
                calc_xyz_controls.layout.display = 'flex'
                calc_coord_controls.layout.display = 'none'
                calc_update_xyz_viewer(initial_load=True)
            else:
                # Single frame
                calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, {len(lines)} lines)'
                _calc_xyz_frames.clear()
                _calc_xyz_frames.extend(parse_xyz_frames(content))
                if _calc_xyz_frames:
                    _calc_xyz_current_frame[0] = 0
                    calc_xyz_frame_input.max = len(_calc_xyz_frames)
                    calc_xyz_frame_input.value = 1
                    calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                    calc_xyz_controls.layout.display = 'flex'
                    calc_coord_controls.layout.display = 'none'
                else:
                    calc_xyz_controls.layout.display = 'none'
                calc_xyz_frame_label.value = ''
                with calc_mol_viewer:
                    view = py3Dmol.view(width=520, height=520)
                    view.addModel(content, 'xyz')
                    view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                    view.zoomTo()
                    view.show()
            calc_render_content(scroll_to='top')
        except Exception as e:
            calc_set_message(f'Error: {e}')

    # PNG image
    elif suffix == '.png':
        try:
            data = full_path.read_bytes()
            b64 = base64.b64encode(data).decode('ascii')
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            calc_content_area.value = (
                "<div style='border:1px solid #ddd; padding:6px; background:#fafafa;'>"
                f"<img src='data:image/png;base64,{b64}' style='max-width:50%; max-height:520px; height:auto; display:block;' />"
                "</div>"
            )
            calc_update_view()
        except Exception as e:
            calc_set_message(f'Error: {e}')

    # Cube volumetric data
    elif suffix in ['.cube', '.cub']:
        calc_view_toggle.disabled = False
        calc_view_toggle.value = False
        calc_update_view()
        try:
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            lower_name = name.lower()
            if lower_name.endswith('.esp.cube') or lower_name.endswith('.esp.cub'):
                from delfin.reporting.esp_report import generate_esp_png_for_state

                workspace_root = None
                search = full_path.parent
                while True:
                    if (search / 'CONTROL.txt').exists():
                        workspace_root = search
                        break
                    if search.parent == search:
                        break
                    search = search.parent
                if workspace_root is None:
                    workspace_root = full_path.parent

                rendered = generate_esp_png_for_state(workspace_root, 'S0')

                calc_set_message('ESP rendered via DELFIN report pipeline. Toggle Visualize to view.')
                _calc_file_content = ''
                with calc_mol_viewer:
                    if rendered and rendered.exists():
                        data = rendered.read_bytes()
                        b64 = base64.b64encode(data).decode('ascii')
                        display(HTML(f"<img src='data:image/png;base64,{b64}' style='max-width:100%; max-height:520px; height:auto; display:block;' />"))
                    else:
                        print('ESP render failed (no PNG generated).')
            else:
                content = full_path.read_text()
                _calc_file_content = ''
                calc_set_message('Cube file loaded. Toggle Visualize to render isosurfaces.')
                with calc_mol_viewer:
                    view = py3Dmol.view(width=520, height=520)
                    view.addModel(content, 'cube')
                    view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                    view.addVolumetricData(content, 'cube', {'isoval': 0.02, 'color': '#0026ff', 'opacity': 0.85})
                    view.addVolumetricData(content, 'cube', {'isoval': -0.02, 'color': '#b00010', 'opacity': 0.85})
                    view.zoomTo()
                    view.show()
        except Exception as e:
            calc_set_message(f'Error: {e}')

    # Binary files
    elif suffix in ['.gbw', '.cis', '.densities', '.tmp']:
        calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
        calc_set_message(f'Binary file ({size_str})\n\nCannot display binary content.')

    # DOCX - convert to HTML with mammoth (preserves layout with inline images)
    elif suffix in ['.doc', '.docx']:
        try:
            import mammoth
            
            docx_name = name
            
            # Convert images to base64 inline
            def convert_image_to_base64(image):
                try:
                    with image.open() as img_stream:
                        img_data = img_stream.read()
                        content_type = image.content_type if image.content_type else 'image/png'
                        b64 = base64.b64encode(img_data).decode('ascii')
                        return {'src': f'data:{content_type};base64,{b64}'}
                except Exception:
                    return {'src': ''}
            
            with open(str(full_path), 'rb') as docx_file:
                result = mammoth.convert_to_html(docx_file, convert_image=mammoth.images.img_element(convert_image_to_base64))
                html_content = result.value
            
            # Wrap in styled container  
            styled_html = (
                '<div style="font-family: Calibri, Arial, sans-serif; line-height: 1.5; padding: 15px; background: white;">'
                '<style>'
                '.docx-content img { max-width: 100%; height: auto; margin: 10px 0; border: 1px solid #ddd; border-radius: 4px; }'
                '.docx-content p { margin: 8px 0; }'
                '.docx-content h1 { color: #1976d2; font-size: 1.8em; margin: 20px 0 10px 0; }'
                '.docx-content h2 { color: #1976d2; font-size: 1.5em; margin: 18px 0 8px 0; }'
                '.docx-content h3 { color: #333; font-size: 1.2em; margin: 15px 0 6px 0; }'
                '.docx-content table { border-collapse: collapse; margin: 10px 0; width: 100%; }'
                '.docx-content td, .docx-content th { border: 1px solid #ddd; padding: 8px; }'
                '.docx-content ul, .docx-content ol { margin: 10px 0; padding-left: 25px; }'
                '</style>'
                f'<div class="docx-content">{html_content}</div>'
                '</div>'
            )
            
            # Extract plain text for copy (simple regex-based extraction)
            import re
            text_only = re.sub(r'<[^>]+>', '', html_content)
            text_only = re.sub(r'\s+', ' ', text_only).strip()
            _calc_file_content = text_only
            
            # Count images
            num_images = html_content.count('<img ')
            img_info = f', {num_images} image{"s" if num_images != 1 else ""}' if num_images > 0 else ''
            
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(docx_name)}</span></b> ({size_str}{img_info})'
            calc_content_area.value = f'<div id="calc-content-box" style="height:520px; overflow-y:auto; overflow-x:hidden; border:1px solid #ddd; background:#fafafa; width:100%; box-sizing:border-box;">{styled_html}</div>'
            calc_copy_btn.disabled = False
            calc_copy_path_btn.disabled = False
            calc_update_view()
            
        except ImportError as ie:
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            calc_set_message('mammoth not installed.\n\nRun: pip install mammoth')
        except Exception as e:
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            calc_set_message(f'Error reading Word file: {e}')

    # Text files - load entire file
    else:
        try:
            content = full_path.read_text()
            _calc_file_content = content
            calc_copy_btn.disabled = False
            calc_copy_path_btn.disabled = False
            lines = content.split('\n')
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str}, {len(lines)} lines)'
            calc_render_content(scroll_to='top')
            if name == 'input.txt':
                calc_view_toggle.disabled = False
                calc_view_toggle.value = False
                calc_update_view()
                try:
                    xyz_content = calc_build_xyz_from_input(content, title=name)
                    if xyz_content:
                        _calc_xyz_frames.clear()
                        _calc_xyz_frames.extend(parse_xyz_frames(xyz_content))
                        if _calc_xyz_frames:
                            _calc_xyz_current_frame[0] = 0
                            calc_xyz_frame_input.max = len(_calc_xyz_frames)
                            calc_xyz_frame_input.value = 1
                            calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                            calc_xyz_controls.layout.display = 'flex'
                            calc_coord_controls.layout.display = 'none'
                        else:
                            calc_xyz_controls.layout.display = 'none'
                        calc_xyz_frame_label.value = ''
                        _calc_xyz_frames.clear()
                        _calc_xyz_frames.extend(parse_xyz_frames(xyz_content))
                        if _calc_xyz_frames:
                            _calc_xyz_current_frame[0] = 0
                            calc_xyz_frame_input.max = len(_calc_xyz_frames)
                            calc_xyz_frame_input.value = 1
                            calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                            calc_xyz_controls.layout.display = 'flex'
                            calc_coord_controls.layout.display = 'none'
                        else:
                            calc_xyz_controls.layout.display = 'none'
                        calc_xyz_frame_label.value = ''
                        with calc_mol_viewer:
                            view = py3Dmol.view(width=520, height=520)
                            view.addModel(xyz_content, 'xyz')
                            view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                            view.zoomTo()
                            view.show()
                except Exception as exc:
                    calc_set_message(f'Error: {exc}')
            elif suffix == '.inp':
                try:
                    xyz_content = calc_build_xyz_from_input(content, title=name)
                    if xyz_content:
                        _calc_xyz_frames.clear()
                        _calc_xyz_frames.extend(parse_xyz_frames(xyz_content))
                        if _calc_xyz_frames:
                            _calc_xyz_current_frame[0] = 0
                            calc_xyz_frame_input.max = len(_calc_xyz_frames)
                            calc_xyz_frame_input.value = 1
                            calc_xyz_frame_total.value = f"<b>/ {len(_calc_xyz_frames)}</b>"
                            calc_xyz_controls.layout.display = 'flex'
                            calc_coord_controls.layout.display = 'none'
                        else:
                            calc_xyz_controls.layout.display = 'none'
                        calc_xyz_frame_label.value = ''
                        calc_view_toggle.disabled = False
                        calc_view_toggle.value = False
                        calc_update_view()
                        with calc_mol_viewer:
                            view = py3Dmol.view(width=520, height=520)
                            view.addModel(xyz_content, 'xyz')
                            view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.25}})
                            view.zoomTo()
                            view.show()
                except Exception as exc:
                    calc_set_message(f'Error: {exc}')
            if suffix == '.inp':
                _calc_selected_inp_path = full_path
                _calc_selected_inp_base = full_path.stem
                calc_recalc_btn.disabled = False
            calc_update_view()
        except UnicodeDecodeError:
            calc_file_info.value = f'<b><span style="word-break:break-all;">{html.escape(name)}</span></b> ({size_str})'
            calc_set_message(f'Binary file ({size_str})\n\nCannot display.')
        except Exception as e:
            calc_set_message(f'Error: {e}')

# Connect event handlers
calc_back_btn.on_click(calc_on_back)
calc_home_btn.on_click(calc_on_home)
calc_refresh_btn.on_click(calc_on_refresh)
calc_top_btn.on_click(calc_go_top)
calc_bottom_btn.on_click(calc_go_bottom)
calc_search_btn.on_click(calc_do_search)
calc_search_input.observe(lambda change: calc_do_search(), names='value')
calc_search_suggest.observe(calc_on_suggest, names='value')
calc_view_toggle.observe(lambda change: calc_update_view(), names='value')
calc_prev_btn.on_click(calc_prev_match)
calc_next_btn.on_click(calc_next_match)
calc_recalc_btn.on_click(calc_on_recalc_click)
calc_submit_recalc_btn.on_click(calc_on_submit_recalc)
calc_delete_btn.on_click(calc_on_delete_click)
calc_delete_yes_btn.on_click(calc_on_delete_yes)
calc_delete_no_btn.on_click(calc_on_delete_no)
calc_file_list.observe(calc_on_select, names='value')
calc_folder_search.observe(calc_filter_file_list, names='value')

def calc_on_sort_change(change):
    '''Re-list directory when sort order changes.'''
    saved_filter = calc_folder_search.value
    calc_list_directory()
    if saved_filter:
        calc_folder_search.value = saved_filter
        calc_filter_file_list()

calc_sort_dropdown.observe(calc_on_sort_change, names='value')

# Initialize
if CALC_DIR.exists():
    calc_list_directory()
else:
    calc_file_list.options = ['(calc folder not found)']

# Build layout
calc_mol_container = widgets.VBox([
    calc_mol_label,
    calc_xyz_frame_label,
    calc_xyz_controls,
    calc_coord_controls,
    calc_mol_viewer
], layout=widgets.Layout(display='none', margin='0 0 10px 0', width='100%', align_items='flex-start'))

calc_nav_bar = widgets.VBox([
    calc_path_label,
    widgets.HBox([calc_back_btn, calc_home_btn, calc_refresh_btn, calc_delete_btn], layout=widgets.Layout(width='100%', overflow_x='hidden', justify_content='flex-start', gap='6px'))
], layout=widgets.Layout(width='100%', overflow_x='hidden'))

calc_content_toolbar = widgets.HBox([
    calc_top_btn, calc_bottom_btn,
    widgets.HTML('&nbsp;│&nbsp;'),
    calc_search_input, calc_search_suggest, calc_search_btn, widgets.HTML('&nbsp;&nbsp;'),
    calc_prev_btn, calc_next_btn, calc_options_dropdown, calc_override_input, calc_override_time, calc_override_btn, calc_search_result
], layout=widgets.Layout(margin='5px 0', width='100%', overflow_x='hidden', gap='6px'))

calc_recalc_toolbar = widgets.HBox([
    calc_recalc_time,
    calc_submit_recalc_btn,
    calc_recalc_status
], layout=widgets.Layout(margin='5px 0', width='100%', overflow_x='hidden', gap='8px', align_items='center'))
calc_recalc_toolbar.layout.display = 'none'

calc_left = widgets.VBox([
    calc_nav_bar,
    calc_filter_sort_row,
    calc_file_list
], layout=widgets.Layout(flex='0 0 260px', min_width='220px', max_width='300px', padding='5px', overflow_x='hidden', overflow_y='hidden'))

calc_css = widgets.HTML(
    '<style>'
    '#calc-content-box { overflow-x:hidden !important; }'
    '.calc-tab, .calc-tab * { overflow-x:hidden !important; box-sizing:border-box; }.calc-tab { overflow:hidden !important; }.calc-right { overflow:hidden !important; }.calc-tab .widget-vbox, .calc-tab .widget-hbox { overflow-y:hidden !important; }.calc-right .widget-output, .calc-right .jupyter-widgets-output-area { overflow:hidden !important; }.calc-right .widget-output .output_area { overflow:hidden !important; }.calc-right .widget-output .output { overflow:hidden !important; }.calc-right .output_scroll, .calc-right .output_subarea, .calc-right .output_wrapper { overflow:hidden !important; }.calc-right .jp-OutputArea, .calc-right .jp-OutputArea-child, .calc-right .jp-OutputArea-output { overflow:hidden !important; }.calc-right .p-Widget { overflow:hidden !important; }.calc-right .widget-textarea { overflow:visible !important; }.calc-right .widget-textarea textarea { overflow-y:auto !important; }'
    '.calc-left, .calc-right { overflow-x:hidden !important; }'
    '.calc-left * { max-width:100% !important; }'
    '.calc-right * { max-width:100% !important; }'
    '.calc-left .widget-select, .calc-left .widget-select select { overflow-x:hidden !important; }'
    '.calc-left .widget-select select { text-overflow: ellipsis; white-space: nowrap; }'
    '.calc-left .widget-text input { overflow-x:hidden !important; overflow-y:hidden !important; text-overflow: ellipsis; }'
    '.calc-right .widget-text input { overflow-x:hidden !important; overflow-y:hidden !important; }'
    '.calc-tab .widget-text { overflow:visible !important; }'
    '.calc-tab input { overflow:hidden !important; height:26px !important; line-height:26px !important; padding:0 6px !important; box-sizing:border-box !important; }'
    '.calc-tab input::-webkit-scrollbar { width:0; height:0; display:none; }'
    '.calc-tab input { scrollbar-width:none; }'
    '.calc-left .widget-vbox { overflow:hidden !important; }'
    '.calc-left code { display:block !important; overflow:hidden !important; text-overflow:ellipsis !important; white-space:nowrap !important; }'
    '</style>'
)

calc_right = widgets.VBox([
    widgets.HBox([
        calc_file_info,
        widgets.HBox([calc_copy_path_btn, calc_copy_btn, calc_report_btn, calc_view_toggle], layout=widgets.Layout(gap='10px'))
    ], layout=widgets.Layout(align_items='center', justify_content='space-between')),
    calc_path_display,
    calc_mol_container,
    calc_content_label,
    calc_delete_confirm,
    calc_delete_status,
    calc_recalc_toolbar,
    calc_content_toolbar,
    calc_override_status,
    calc_content_area,
    calc_edit_area
], layout=widgets.Layout(flex='1 1 0', min_width='0', padding='5px', overflow_x='hidden', overflow_y='hidden'))

tab5_content = widgets.VBox([
    calc_css,
    widgets.HTML('<h3>📂 Calculations Browser</h3>'),
    widgets.HBox([calc_left, calc_right], layout=widgets.Layout(width='100%', overflow_x='hidden', align_items='stretch', gap='16px'))
], layout=widgets.Layout(padding='10px', overflow_x='hidden', width='100%', max_width='100%', height='780px'))
tab5_content.add_class('calc-tab')
calc_left.add_class('calc-left')
calc_right.add_class('calc-right')



# === Repo Update (git pull) ===


def coord_to_xyz(coord_text):
    lines = coord_text.strip().split('\n')
    xyz_lines = []
    bohr_to_ang = 0.529177249
    in_coord = False
    for line in lines:
        if line.strip().startswith('$coord'): in_coord = True; continue
        if line.strip().startswith('$'): in_coord = False; continue
        if in_coord:
            parts = line.split()
            if len(parts) >= 4:
                try:
                    x, y, z = float(parts[0])*bohr_to_ang, float(parts[1])*bohr_to_ang, float(parts[2])*bohr_to_ang
                    xyz_lines.append(f"{parts[3].capitalize()}  {x:12.6f}  {y:12.6f}  {z:12.6f}")
                except: continue
    if not xyz_lines:
        for line in lines:
            parts = line.split()
            if len(parts) >= 4 and parts[0][0].isalpha():
                try: float(parts[1]); xyz_lines.append(line)
                except: continue
    if not xyz_lines: return None
    return f"{len(xyz_lines)}\nTM Preview\n" + "\n".join(xyz_lines)

def update_tm_molecule_view(change=None):
    with tm_mol_output:
        clear_output(wait=True)
        raw = tm_coords.value.strip()




# === Layout (Dashboard style) ===
spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))
spacer_large = widgets.Label(value='', layout=widgets.Layout(height='20px'))

submit_left = widgets.VBox([
    job_name_widget, spacer, job_type_widget, custom_time_widget, spacer_large,
    widgets.HTML('<b>Input (XYZ or SMILES):</b>'), coords_widget, spacer, widgets.HBox([convert_smiles_button, build_complex_button], layout=widgets.Layout(gap='10px')),
    spacer_large, widgets.HTML('<b>Batch SMILES:</b>'), smiles_batch_widget, spacer, widgets.HBox([smiles_prev_button, smiles_preview_label, smiles_next_button, submit_smiles_list_button], layout=widgets.Layout(gap='2px', align_items='center')), smiles_batch_output,
    spacer_large, widgets.HTML('<b>CONTROL.txt:</b>'), control_widget, spacer,
    widgets.HBox([validate_button, submit_button]), output_area, validate_output
], layout=widgets.Layout(width='50%', padding='10px'))

submit_right = widgets.VBox([
    widgets.HTML('<b>Molecule Preview:</b>'), mol_output,
    spacer_large, widgets.HTML('<b>Only GOAT:</b>'),
    widgets.HBox([only_goat_charge, only_goat_solvent, only_goat_submit_button]),
    only_goat_output
], layout=widgets.Layout(width='50%', padding='10px'))

tab1_content = widgets.HBox([submit_left, submit_right])

recalc_refresh_btn = widgets.Button(description='REFRESH FOLDERS', button_style='info', layout=widgets.Layout(width='150px'))
recalc_refresh_btn.on_click(lambda b: refresh_recalc_folders())

tab2_content = widgets.VBox([
    widgets.HTML('<h3>Recalc DELFIN Job</h3>'),
    widgets.HTML('<p>Select a job folder, edit CONTROL.txt, and resubmit.</p>'),
    widgets.HBox([recalc_folder_dropdown, recalc_refresh_btn]),
    recalc_time_limit,
    recalc_custom_time_widget,
    recalc_control_widget, spacer,
    recalc_button, recalc_output
], layout=widgets.Layout(padding='10px'))

tab3_content = widgets.VBox([
    widgets.HTML('<h3>Job Status</h3>'),
    widgets.HTML('<p>Select a job and click CANCEL to cancel it.</p>'),
    job_table_html, job_start_html,
    widgets.HBox([job_dropdown, cancel_button, refresh_button], layout=widgets.Layout(margin='10px 0')),
    job_status_output
], layout=widgets.Layout(padding='10px'))

orca_left = widgets.VBox([
    orca_row1,
    orca_row2,
    orca_row3,
    orca_row4,
    orca_row5,
    orca_row6,
    orca_row6b,
    orca_row_solv,
    orca_row_opts,
    orca_row7,
    orca_row8,
    orca_row9,
    orca_row_upload,
    orca_row_paths,
    orca_row_buttons,
    orca_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_right = widgets.VBox([
    widgets.HTML('<b>ORCA Input Preview (editable):</b>'),
    orca_preview,
    widgets.HTML('<b>Molecule Preview:</b>', layout=widgets.Layout(margin='10px 0 0 0')),
    widgets.HBox([orca_mol_output], layout=widgets.Layout(justify_content='center', width='100%'))
], layout=widgets.Layout(width='50%', padding='10px'))

tab4_content = widgets.VBox([
    widgets.HTML('<h3>ORCA Input Builder</h3>'),
    widgets.HTML("<a href=\"https://orca-manual.mpi-muelheim.mpg.de/\" target=\"_blank\">ORCA User Manual</a>"),
    widgets.HBox([orca_left, orca_right])
], layout=widgets.Layout(padding='10px'))


# === Repo Update (git pull) ===
busy_css = widgets.HTML(
    "<style>"
    ".delfin-busy { display:inline-block; width:14px; height:14px; border:2px solid #1976d2; border-top-color: transparent; border-radius:50%; animation: delfin-spin 0.9s linear infinite; }"
    "@keyframes delfin-spin { from { transform: rotate(0deg); } to { transform: rotate(360deg); } }"
    ".widget-textarea textarea { resize: none !important; }"
    "textarea { resize: none !important; }"
    "</style>"
)

busy_indicator = widgets.HTML(value="")

def set_delfin_busy(is_busy):
    busy_indicator.value = "<span class='delfin-busy' title='Working'></span>" if is_busy else ""

pull_delfin_btn = widgets.Button(
    description='PULL DELFIN',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

orca_version_dropdown = widgets.Dropdown(
    options=[('ORCA 6.1.1', '/opt/orca')],
    value='/opt/orca',
    description='ORCA:',
    layout=widgets.Layout(width='170px'),
    style={'description_width': 'initial'},
    disabled=True
)

pull_delfin_output = widgets.Output()

_repo_dir = _deflin_root if _deflin_root else None

def handle_pull_delfin(button):
    with pull_delfin_output:
        clear_output()
        if not _repo_dir or not Path(_repo_dir).exists():
            print(f'Repo path not found: {_repo_dir}')
            return
        print(f'Running git pull in {_repo_dir} ...')
        set_delfin_busy(True)
        try:
            result = subprocess.run(
                ['git', '-C', str(_repo_dir), 'pull'],
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                check=False
            )
            print(result.stdout.strip() or '(no output)')
        except Exception as e:
            print(f'Error: {e}')
        finally:
            set_delfin_busy(False)

pull_delfin_btn.on_click(handle_pull_delfin)


# === Create Tabs (5 tabs, no TURBOMOLE) ===
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content, tab5_content])
tabs.set_title(0, 'Submit Job')
tabs.set_title(1, 'Recalc')
tabs.set_title(2, 'Job Status')
tabs.set_title(3, 'ORCA Builder')
tabs.set_title(4, 'Calculations')

# Auto-refresh job list when switching to Job Status tab
def _on_tab_change(change):
    if change['new'] == 2:  # Job Status tab
        refresh_job_list()

tabs.observe(_on_tab_change, names='selected_index')

display(widgets.VBox([
    busy_css,
    widgets.HBox([
        widgets.HTML('<h2 style="color:#1976d2; margin:0;">DELFIN Dashboard (Local)</h2>'),
        widgets.HBox([busy_indicator, pull_delfin_btn, orca_version_dropdown], layout=widgets.Layout(margin='0 0 0 12px', align_items='center', gap='8px')),
    ], layout=widgets.Layout(align_items='center', justify_content='space-between', width='100%')),
    pull_delfin_output
], layout=widgets.Layout(width='100%')))
display(widgets.VBox([_js_output, tabs]))
